In [1]:
from pyomo.environ import *

In [2]:
def solve_pyomo(para):
    p_ei  = para['pei']
    p_er  = para['per']
    p_ea  = 1-p_ei-p_er
    p_ai  = p_ei
    p_ar  = p_er
    p_irr = 1/8
    alpha  = 0.6
    beta_1 = para['beta1']
    beta_2 = para['beta2']
    cost_tr = 0.0977
    p_id  = 1.1/1000
    p_idd = 0.1221/1000
    p_srr = 0.7/3
    S = 100000
    E = 20
    A = 0
    I = 0
    R = 0
    D = 0
    N = S+E+A+I+R+D
    K = 50
    w_max  = 1.0
    tr_max = 1.0
    v_max = 0.2/14
    M = 3.6e3
    
    m = ConcreteModel()
    m.k = RangeSet(0, K)
    m.t = RangeSet(0, K-1)
    m.N = N
    m.tri = Var(m.k, bounds=(0, 1)) 
    m.wi = Var(m.k, bounds=(0, 1))  
    m.v = Var(m.k, bounds=(0, 1))
    m.aq = Var(m.k, bounds=(0, 1))
    m.iq = Var(m.k, bounds=(0, 1))
    m.x_s = Var(m.k, domain = NonNegativeReals)
    m.x_e = Var(m.k, domain = NonNegativeReals)
    m.x_a = Var(m.k, domain = NonNegativeReals)
    m.x_i = Var(m.k, domain = NonNegativeReals)
    m.x_r = Var(m.k, domain = NonNegativeReals)
    m.x_d = Var(m.k, domain = NonNegativeReals)
    
    m.xs_c = Constraint(m.t, rule=lambda m, k: m.x_s[k+1] == m.x_s[k]
                    -(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                    *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                    -p_srr*m.v[k]*v_max*m.x_s[k])

    m.xe_c = Constraint(m.t, rule=lambda m, k: m.x_e[k+1] == m.x_e[k]
                        +alpha*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        -p_ei*m.x_e[k]
                        -p_er*m.x_e[k]
                        -m.wi[k]*p_ea*m.x_e[k])
                     
    m.xa_c = Constraint(m.t, rule=lambda m, k: m.x_a[k+1] == m.x_a[k]
                        +m.wi[k]*p_ea*m.x_e[k]
                        -p_ai*m.x_a[k]
                        -p_ar*m.x_a[k])
                                        
    m.xi_c = Constraint(m.t, rule=lambda m, k: m.x_i[k+1] == m.x_i[k]
                        +(1-alpha)*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +p_ei*m.x_e[k]
                        +p_ai*m.x_a[k]
                        -m.tri[k]*p_irr*m.x_i[k]
                        -(1-m.tri[k])*m.x_i[k]*p_id
                        -m.tri[k]*m.x_i[k]*p_idd)
                                                                                       
    m.xr_c = Constraint(m.t, rule=lambda m, k: m.x_r[k+1] == m.x_r[k]
                        +m.tri[k]*p_irr*m.x_i[k]
                        +p_er*m.x_e[k]
                        +p_ar*m.x_a[k]
                        +p_srr*m.v[k]*v_max*m.x_s[k])

    m.xd_c = Constraint(m.t, rule=lambda m, k: m.x_d[k+1] == m.x_d[k]
                        +(1-m.tri[k])*m.x_i[k]*p_id
                        +m.tri[k]*m.x_i[k]*p_idd)
    
    m.pc = ConstraintList()
    m.pc.add(m.x_s[0]==S)
    m.pc.add(m.x_e[0]==E)
    m.pc.add(m.x_a[0]==A)
    m.pc.add(m.x_i[0]==I)
    m.pc.add(m.x_r[0]==R)
    m.pc.add(m.x_d[0]==D)

    gamma = 1
    m.sumcost = sum(m.wi[k]*(0.000369*(m.wi[k]*(m.x_s[k]+m.x_a[k]+m.x_e[k]+m.x_r[k])/m.N)
                         *(m.wi[k]*(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])/m.N) + 0.001057)
                         *(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])
                    +0.0977*m.tri[k]*m.x_i[k]+m.x_s[k]
                    *m.v[k]*v_max*0.07 + m.x_i[k]*m.iq[k]*0.02 + m.x_a[k]*m.aq[k]*0.02 for k in m.t) 
    m.budget_c = Constraint(expr = m.sumcost <= M)

    m.suminfected = sum(2.6 * (beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +(m.x_d[k+1] - m.x_d[k]) * 60 for k in m.t)
    m.obj = Objective(expr = m.suminfected, sense = minimize)
    solver = SolverFactory('ipopt')
    solver.options['max_iter']= 100000
    try:
        solver.solve(m,tee=True)
    except:
        print()

In [3]:
import pickle
interval = 0.6
para_truth = pickle.load(open('para_truth' + str(round(interval * 100)), 'rb'))

In [4]:
solve_pyomo(para_truth[0])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 303  4.1650833e+05 4.37e+02 1.43e+04  -1.0 2.34e+03  -2.5 2.13e-03 6.13e-04h  1
 304  4.1651079e+05 4.37e+02 1.43e+04  -1.0 1.40e+03  -2.1 1.15e-02 5.77e-04h  1
 305  4.1651096e+05 4.37e+02 1.43e+04  -1.0 3.31e+03  -2.6 8.44e-03 3.71e-04h  1
 306  4.1650497e+05 4.37e+02 1.42e+04  -1.0 4.89e+03  -3.1 2.35e-03 7.77e-04f  1
 307  4.1650449e+05 4.37e+02 1.42e+04  -1.0 4.82e+03  -2.6 4.57e-03 1.71e-04h  1
 308  4.1649905e+05 4.36e+02 1.42e+04  -1.0 7.09e+03  -3.1 6.82e-03 5.33e-04f  1
 309  4.1649736e+05 4.36e+02 1.42e+04  -1.0 6.57e+03  -2.7 3.36e-03 2.11e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310  4.1648882e+05 4.36e+02 1.42e+04  -1.0 1.12e+04  -3.2 3.92e-03 5.27e-04f  1
 311  4.1648450e+05 4.36e+02 1.42e+04  -1.0 9.33e+03  -2.7 3.56e-03 4.12e-04f  1
 312  4.1647813e+05 4.36e+02 1.42e+04  -1.0 1.71e+04  -3.2 3.17e-03 3.27e-04f  1
 313  4.1647300e+05 4.36e+02 1.42e+04  -1.0 1.33e+04  -2.8 3.51e-03 3.98e-04f  1
 314  4.1646197e+05 4.35e+02

 412  3.4702539e+05 1.67e+02 8.91e+03  -1.0 3.90e+04  -5.6 4.04e-02 1.20e-02f  1
 413  3.4681387e+05 1.52e+02 6.68e+03  -1.0 6.22e+03  -5.2 2.30e-01 1.01e-01f  1
 414  3.4601294e+05 1.07e+02 1.08e+03  -1.0 5.71e+03  -4.8 8.09e-01 3.97e-01f  1
 415  3.4582144e+05 9.50e+01 1.10e+03  -1.0 3.82e+03  -5.2 3.87e-02 1.15e-01f  1
 416  3.4408927e+05 1.11e+02 1.17e+03  -1.0 3.42e+03  -4.8 1.65e-01 9.90e-01f  1
 417  3.4404327e+05 1.07e+02 1.24e+03  -1.0 7.77e+03  -5.3 1.62e-02 4.11e-02f  1
 418  3.4404102e+05 1.07e+02 1.70e+03  -1.0 3.69e+03  -4.9 1.42e-01 7.64e-03f  1
 419  3.4384631e+05 9.87e+01 9.84e+02  -1.0 9.47e+03  -5.3 2.54e-03 9.93e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 420  3.4384304e+05 9.83e+01 2.03e+03  -1.0 5.19e+03  -4.9 1.19e-01 4.34e-03f  1
 421  3.4380000e+05 7.36e+01 3.09e+03  -1.0 1.70e+03  -4.5 1.78e-02 2.66e-01f  1
 422  3.4370977e+05 6.99e+01 9.21e+02  -1.0 8.60e+03  -5.0 1.35e-01 5.56e-02f  1
 423  3.4361982e+05 6.00e+01

 506  3.3557854e+05 1.45e-03 5.57e-02  -2.5 3.45e-01  -0.8 1.00e+00 1.00e+00f  1
 507  3.3557749e+05 2.85e-04 5.96e+04  -3.8 9.65e-01  -1.3 1.00e+00 7.77e-01f  1
 508  3.3557680e+05 9.73e-04 4.42e+04  -3.8 3.08e+00  -1.7 3.43e-01 2.58e-01f  1
 509  3.3557650e+05 3.07e-03 3.10e+04  -3.8 1.17e+00  -1.3 5.54e-01 2.98e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  3.3557614e+05 3.07e-03 2.65e+04  -3.8 3.51e+00  -1.8 3.77e-01 1.46e-01f  1
 511  3.3557540e+05 2.81e-03 2.37e+04  -3.8 1.05e+01  -2.3 1.00e+00 1.06e-01f  1
 512  3.3556892e+05 1.67e-03 1.62e+04  -3.8 3.15e+01  -2.8 1.00e+00 3.14e-01f  1
 513  3.3555584e+05 3.42e-03 1.28e+04  -3.8 9.44e+01  -3.2 5.24e-01 2.12e-01f  1
 514  3.3555293e+05 3.26e-04 5.58e-02  -3.8 4.42e+00  -1.9 1.00e+00 1.00e+00f  1
 515  3.3554429e+05 1.44e-03 5.58e-02  -3.8 1.33e+01  -2.4 1.00e+00 1.00e+00f  1
 516  3.3553344e+05 3.11e-03 3.90e+00  -3.8 3.99e+01  -2.9 1.00e+00 4.18e-01f  1
 517  3.3549152e+05 3.51e-02

 599  3.2892762e+05 6.55e-04 1.64e+03  -3.8 5.78e-02  -0.0 1.00e+00 6.22e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 600  3.2892752e+05 3.39e-04 8.73e+02  -3.8 1.58e-01  -0.5 1.00e+00 4.86e-01f  1
 601  3.2892744e+05 6.26e-04 7.41e+02  -3.8 4.76e-01  -1.0 2.41e-01 1.51e-01f  1
 602  3.2892702e+05 5.86e-04 7.54e+02  -3.8 1.41e+00  -1.4 5.46e-01 2.50e-01f  1
 603  3.2892698e+05 1.69e-02 5.91e+02  -3.8 1.26e+00  -1.0 1.00e+00 1.36e-01f  2
 604  3.2892684e+05 8.29e-03 4.67e+02  -3.8 2.01e-01  -0.6 1.00e+00 4.89e-01f  1
 605  3.2892670e+05 1.72e-02 3.64e+02  -3.8 6.53e-01  -1.1 2.80e-01 2.38e-01f  1
 606  3.2892643e+05 1.51e-02 7.36e+02  -3.8 1.79e+00  -1.5 7.91e-01 1.32e-01f  1
 607  3.2892589e+05 1.38e-02 7.40e+02  -3.8 5.48e+00  -2.0 3.93e-01 9.22e-02f  1
 608  3.2892584e+05 1.17e-02 8.64e+02  -3.8 2.54e-01  -0.7 1.00e+00 1.56e-01h  1
 609  3.2892501e+05 2.41e-04 5.17e-02  -3.8 7.62e-01  -1.2 1.00e+00 1.00e+00f  1
iter    objective    inf_pr 

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1900  1.0286577e+04 1.13e+01 4.51e+03  -3.8 1.55e+04    -  3.38e-02 1.35e-02f  1
1901  9.8130708e+03 1.11e+01 5.51e+03  -3.8 1.34e+04    -  2.57e-02 1.22e-02f  1
1902  9.7966277e+03 1.11e+01 7.17e+03  -3.8 1.21e+04    -  2.36e-02 4.77e-04f  1
1903  7.6289009e+03 1.11e+01 3.33e+03  -3.8 1.16e+04    -  4.36e-06 6.50e-02f  1
1904  7.3532816e+03 1.10e+01 3.52e+03  -3.8 9.37e+03    -  5.31e-03 8.65e-03f  1
1905  7.3138283e+03 1.10e+01 5.55e+03  -3.8 8.75e+03    -  3.50e-02 1.61e-03f  1
1906  5.8298854e+03 1.09e+01 2.17e+03  -3.8 8.42e+03    -  5.70e-06 6.47e-02f  1
1907  5.5125178e+03 1.09e+01 2.48e+03  -3.8 1.26e+04    -  6.28e-03 1.09e-02f  1
1908  5.5013251e+03 1.08e+01 2.45e+03  -3.8 6.08e+03    -  1.35e-05 6.46e-04f  1
1909  5.0916871e+03 1.26e+01 2.78e+03  -3.8 6.35e+03    -  3.22e-02 2.16e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1910  5.0915675e+03 1.26e+01

2004  1.0216286e+03 1.37e+00 5.43e+04  -3.8 7.64e+02  -3.9 1.87e-02 1.79e-04f  1
2005  1.0142774e+03 1.37e+00 5.39e+04  -3.8 3.57e+03  -4.4 1.73e-05 1.60e-03f  1
2006  1.0141982e+03 1.37e+00 5.42e+04  -3.8 9.47e+02  -4.0 4.53e-03 2.76e-05f  1
2007  9.9940119e+02 1.37e+00 5.38e+04  -3.8 3.43e+03  -4.4 2.03e-03 3.24e-03f  1
2008  9.9939499e+02 1.37e+00 5.46e+04  -3.8 1.17e+03  -4.0 6.57e-03 2.22e-06f  1
2009  9.0086909e+02 1.40e+00 5.00e+04  -3.8 3.86e+03  -4.5 3.49e-03 2.03e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2010  9.0086019e+02 1.40e+00 5.07e+04  -3.8 1.09e+03  -4.1 6.83e-03 3.27e-06f  1
2011  7.9058589e+02 1.51e+00 4.55e+04  -3.8 3.71e+03  -4.5 2.51e-03 2.49e-02f  1
2012  7.9057277e+02 1.51e+00 4.84e+04  -3.8 9.99e+02  -4.1 4.18e-02 5.28e-06f  1
2013  6.7298640e+02 1.89e+00 4.34e+04  -3.8 3.31e+03  -4.6 7.49e-03 3.31e-02f  1
2014  6.7292296e+02 1.89e+00 5.26e+04  -3.8 1.03e+03  -4.2 9.85e-02 3.35e-05f  1
2015  6.6165384e+02 1.92e+00

2099  2.9844638e+02 3.91e+00 6.94e+03  -3.8 1.01e+02  -3.6 2.88e-02 2.38e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2100  2.9626115e+02 3.64e+00 2.44e+04  -3.8 1.38e+01  -2.3 4.56e-01 7.17e-02f  1
2101  2.9537352e+02 3.53e+00 2.37e+04  -3.8 3.05e+01  -2.8 8.69e-02 3.04e-02f  1
2102  2.9328206e+02 3.52e+00 2.44e+04  -3.8 1.81e+03    -  6.04e-06 1.99e-03f  1
2103  2.9326563e+02 3.52e+00 2.44e+04  -3.8 2.32e+04    -  5.54e-06 2.28e-06f  1
2104  2.9323922e+02 3.52e+00 2.05e+04  -3.8 5.49e+01  -3.2 1.72e-01 6.25e-04f  1
2105  2.9151431e+02 3.51e+00 2.10e+04  -3.8 1.73e+03    -  4.09e-06 1.60e-03f  1
2106  2.6863041e+02 3.80e+00 3.62e+04  -3.8 1.42e+03    -  4.71e-04 4.51e-02f  1
2107  2.6858058e+02 3.80e+00 3.59e+04  -3.8 1.23e+02  -3.7 2.47e-02 7.91e-04f  1
2108  2.6806397e+02 3.72e+00 3.61e+04  -3.8 2.24e+01  -2.4 9.87e-02 1.91e-02f  1
2109  2.6689599e+02 3.64e+00 3.53e+04  -3.8 5.12e+01  -2.9 1.12e-03 1.66e-02f  1
iter    objective    inf_pr 

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2200  1.2881484e+02 1.43e+01 4.83e+04  -3.8 6.35e+02  -5.8 8.86e-05 1.39e-02f  1
2201  1.2642381e+02 1.41e+01 4.06e+04  -3.8 1.24e+03  -6.3 5.53e-02 2.11e-02f  1
2202  1.2339939e+02 1.38e+01 1.83e+04  -3.8 8.71e+02    -  1.29e-01 2.38e-02f  1
2203  1.2339820e+02 1.38e+01 1.49e+04  -3.8 2.15e+02    -  1.55e-01 4.66e-05h  1
2204  1.2305404e+02 1.37e+01 1.50e+04  -3.8 2.11e+02    -  2.44e-05 1.13e-02f  1
2205  1.2292565e+02 1.22e+01 1.43e+04  -3.8 7.15e+01    -  5.50e-02 1.06e-01h  1
2206  1.2283263e+02 1.20e+01 1.30e+04  -3.8 4.04e+03  -6.8 6.44e-02 1.56e-02h  1
2207  1.2269268e+02 1.06e+01 1.17e+04  -3.8 4.21e+01    -  9.95e-02 1.17e-01h  1
2208  1.2233909e+02 1.06e+01 1.18e+04  -3.8 4.98e+02    -  1.68e-05 4.41e-03f  1
2209  1.2225705e+02 9.47e+00 9.80e+03  -3.8 4.16e+01    -  1.62e-01 1.06e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2210  1.1702263e+02 8.49e+00

2306  6.7527518e+01 1.71e+01 4.81e+02  -3.8 6.44e+02    -  1.25e-01 3.13e-01h  1
2307  6.7525767e+01 1.18e+01 4.03e+02  -3.8 1.79e+02    -  1.30e-01 4.06e-01h  1
2308  6.7525602e+01 1.45e+00 3.39e+01  -3.8 5.56e-02  -0.6 1.00e+00 8.81e-01h  1
2309  6.7512613e+01 1.84e+00 9.55e+01  -3.8 5.25e+02    -  1.96e-01 1.21e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2310  6.7486613e+01 7.07e-01 6.08e+02  -3.8 4.65e+02    -  1.94e-01 5.06e-01h  1
2311  6.7491012e+01 1.21e-07 3.37e-01  -3.8 2.69e-02  -1.1 1.00e+00 1.00e+00h  1
2312  6.7480472e+01 2.93e-02 1.29e+02  -3.8 5.36e+02    -  1.54e-01 2.13e-01h  1
2313  6.7466166e+01 1.03e-02 3.90e+02  -3.8 7.68e+02    -  1.64e-01 2.82e-01f  1
2314  6.7468847e+01 6.42e-07 2.60e-01  -3.8 8.37e-02  -1.6 1.00e+00 1.00e+00h  1
2315  6.7468398e+01 5.86e-06 5.28e+00  -3.8 6.71e-01  -2.0 9.33e-02 6.25e-02H  1
2316  6.7467818e+01 2.85e-07 7.07e-01  -3.8 1.15e-01  -1.6 1.00e+00 1.00e+00F  1
2317  6.7467827e+01 4.23e-07

In [5]:
solve_pyomo(para_truth[1])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 193r 7.7574169e+03 9.81e+04 4.13e+01  -4.5 3.27e+01   0.1 3.37e-02 2.30e-01f  1
 194r 7.8840942e+03 9.80e+04 1.56e+02  -4.5 9.74e+01  -0.4 6.83e-01 4.28e-01f  1
 195r 8.4653343e+03 9.79e+04 1.06e+03  -4.5 2.87e+02  -0.9 2.33e-01 6.65e-01f  1
 196r 8.7962227e+03 9.77e+04 4.01e+01  -4.5 1.07e+02  -0.4 1.00e+00 1.00e+00f  1
 197r 8.9195524e+03 9.77e+04 4.00e+01  -4.5 4.02e+01  -0.0 1.00e+00 1.00e+00f  1
 198r 8.9478150e+03 9.77e+04 1.29e+03  -4.5 1.19e+02  -0.5 9.44e-01 7.70e-02f  1
 199r 8.9503459e+03 9.77e+04 3.71e+03  -4.5 3.51e+02  -1.0 5.40e-01 2.33e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200r 9.0612736e+03 9.77e+04 1.26e+03  -4.5 1.34e+02  -0.5 1.00e+00 2.69e-01f  1
 201r 9.1826070e+03 9.76e+04 1.26e+03  -4.5 3.93e+02  -1.0 1.62e-01 9.95e-02f  1
 202r 9.4785160e+03 9.75e+04 4.12e+02  -4.5 1.50e+02  -0.6 1.00e+00 6.41e-01f  1
 203r 9.4884805e+03 9.75e+04 3.99e+02  -4.5 5.62e+01  -0.2 5.22e-01 5.75e-02f  1
 204r 9.4885952e+03 9.75e+04

 483  1.9669403e+05 2.27e+04 6.75e+04  -1.0 2.27e+04  -1.0 3.46e-05 3.24e-04h  1
 484  1.9669609e+05 2.27e+04 8.41e+04  -1.0 2.27e+04  -0.5 3.80e-04 2.19e-05h  1
 485  1.9672577e+05 2.27e+04 1.01e+05  -1.0 2.27e+04  -1.0 1.40e-03 3.65e-04h  1
 486  1.9672941e+05 2.27e+04 1.46e+05  -1.0 2.27e+04  -0.6 9.99e-04 4.56e-05h  1
 487  1.9675816e+05 2.27e+04 1.63e+05  -1.0 2.27e+04  -1.1 1.42e-03 4.04e-04h  1
 488  1.9676569e+05 2.27e+04 2.27e+05  -1.0 2.27e+04  -0.6 1.55e-03 1.04e-04h  1
 489  1.9679272e+05 2.27e+04 2.41e+05  -1.0 2.27e+04  -1.1 1.32e-03 4.00e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  1.9680110e+05 2.27e+04 2.44e+05  -1.0 2.27e+04  -0.7 1.74e-04 1.24e-04h  1
 491  1.9682122e+05 2.27e+04 2.46e+05  -1.0 2.27e+04  -1.2 4.78e-04 3.19e-04h  1
 492  1.9682287e+05 2.27e+04 3.05e+05  -1.0 2.27e+04  -0.7 1.53e-03 2.55e-05h  1
 493  1.9686013e+05 2.27e+04 3.07e+05  -1.0 2.27e+04  -1.2 8.04e-04 6.27e-04h  1
 494  1.9686179e+05 2.27e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 580  2.4915141e+05 9.21e+02 5.03e+05  -1.0 1.10e+03  -2.0 3.55e-03 1.47e-02h  1
 581  2.4918293e+05 9.15e+02 5.79e+05  -1.0 1.08e+03  -1.6 1.38e-01 6.24e-03h  1
 582  2.4922011e+05 9.08e+02 5.75e+05  -1.0 1.09e+03  -2.1 7.20e-03 7.78e-03h  1
 583  2.4924799e+05 9.03e+02 5.48e+05  -1.0 1.19e+03  -2.6 6.67e-02 5.59e-03h  1
 584  2.4934809e+05 8.85e+02 5.62e+05  -1.0 1.22e+03  -2.1 1.15e-01 1.94e-02h  1
 585  2.4954289e+05 8.50e+02 5.48e+05  -1.0 1.24e+03  -2.6 5.26e-03 4.00e-02h  1
 586  2.4962969e+05 8.35e+02 5.38e+05  -1.0 1.18e+03  -2.2 1.78e-02 1.77e-02h  1
 587  2.4963658e+05 8.34e+02 5.44e+05  -1.0 1.03e+04  -1.8 1.35e-04 1.21e-03h  1
 588  2.4966208e+05 8.30e+02 5.44e+05  -1.0 1.21e+03  -2.2 1.26e-01 5.34e-03h  1
 589  2.4967723e+05 8.26e+02 5.43e+05  -1.0 3.67e+03  -2.7 2.94e-03 3.73e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 590  2.4968904e+05 8.24e+02

 675  2.5122285e+05 6.04e+01 1.08e+06  -1.0 2.87e+02  -1.3 2.14e-02 8.84e-03h  1
 676  2.5122451e+05 6.00e+01 1.01e+06  -1.0 2.35e+02  -0.8 2.08e-05 6.17e-03h  1
 677  2.5122629e+05 5.98e+01 7.93e+05  -1.0 2.03e+02   0.5 2.83e-04 3.22e-03h  1
 678  2.5122631e+05 5.98e+01 1.35e+06  -1.0 1.79e+02   0.9 3.98e-03 4.62e-05h  1
 679  2.5123157e+05 5.92e+01 4.79e+05  -1.0 2.35e+02   0.5 6.45e-05 9.73e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 680  2.5123368e+05 5.90e+01 3.30e+05  -1.0 2.21e+02   0.9 3.10e-03 3.83e-03h  1
 681  2.5123555e+05 5.88e+01 6.06e+05  -1.0 2.12e+02   1.3 3.93e-03 3.37e-03h  1
 682  2.5123610e+05 5.87e+01 1.76e+05  -1.0 2.81e+02   0.8 7.01e-05 1.31e-03h  1
 683  2.5123972e+05 5.82e+01 8.43e+05  -1.0 2.36e+02   0.3 2.03e-03 8.12e-03h  1
 684  2.5123994e+05 5.82e+01 3.59e+05  -1.0 3.02e+02  -0.1 1.75e-02 5.36e-04h  1
 685  2.5124138e+05 5.80e+01 3.15e+05  -1.0 2.98e+02  -0.6 5.21e-04 3.72e-03h  1
 686  2.5124159e+05 5.80e+01

...

1542  4.6224905e+03 2.04e+01 3.14e+04  -3.8 4.87e+01  -1.7 2.17e-01 2.88e-02f  1
1543  4.6218194e+03 1.93e+01 3.05e+04  -3.8 1.42e+01  -0.4 3.01e-04 5.25e-02h  1
1544  4.6217080e+03 1.89e+01 2.98e+04  -3.8 1.37e+01   0.9 2.77e-02 2.28e-02h  1
1545  4.6216777e+03 1.86e+01 2.96e+04  -3.8 1.29e+01   1.4 8.42e-04 1.32e-02h  1
1546  4.6213550e+03 1.69e+01 2.72e+04  -3.8 1.27e+01   0.9 6.73e-02 9.38e-02h  1
1547  4.6212454e+03 1.60e+01 2.62e+04  -3.8 1.15e+01   1.3 2.27e-02 5.22e-02h  1
1548  4.6211471e+03 1.54e+01 2.50e+04  -3.8 1.09e+01   0.8 4.96e-02 3.88e-02h  1
1549  4.6211390e+03 1.53e+01 2.47e+04  -3.8 1.04e+01   1.3 1.60e-02 6.23e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1550  4.6208557e+03 1.36e+01 2.33e+04  -3.8 1.04e+01   0.8 4.26e-03 1.11e-01f  1
1551  4.6208349e+03 1.33e+01 2.11e+04  -3.8 9.21e+00   1.2 1.53e-01 2.38e-02h  1
1552  4.6206523e+03 1.25e+01 2.05e+04  -3.8 9.02e+00   0.7 3.51e-03 5.42e-02h  1
1553  4.6206319e+03 1.23e+01

1699  1.3667681e+03 2.63e+01 7.72e+03  -3.8 4.79e+03  -4.4 4.91e-06 2.44e-08h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1700  1.3666582e+03 2.63e+01 7.67e+03  -3.8 5.34e+03  -4.9 3.13e-04 6.41e-06f  1
1701  1.3666016e+03 2.63e+01 5.30e+03  -3.8 1.46e+02  -2.6 1.28e-01 9.34e-05f  1
1702  1.3617200e+03 2.62e+01 3.46e+03  -3.8 3.68e+02  -3.1 1.06e-01 3.49e-03f  1
1703  1.3238004e+03 2.63e+01 3.26e+03  -3.8 9.41e+02  -3.6 6.79e-06 1.23e-02f  1
1704  1.3237990e+03 2.63e+01 1.09e+04  -3.8 1.03e+01   2.3 9.54e-04 9.31e-05h  1
1705  1.3238007e+03 2.63e+01 1.17e+04  -3.8 4.08e+01   1.8 5.66e-05 5.05e-05h  1
1706  1.3238006e+03 2.63e+01 2.87e+04  -3.8 2.10e+00   3.1 7.37e-04 1.42e-05h  1
1707  1.3238003e+03 2.63e+01 5.18e+04  -3.8 2.09e+00   2.6 4.79e-04 4.95e-05h  1
1708  1.3237928e+03 2.63e+01 3.32e+04  -3.8 2.49e+00   2.2 3.93e-04 1.33e-03h  1
1709  1.3237941e+03 2.61e+01 3.26e+04  -3.8 1.95e+00   1.7 1.82e-05 5.84e-03h  1
iter    objective    inf_pr 

1853  5.8899795e+02 4.27e-06 3.10e+02  -3.8 4.97e-01  -0.7 1.00e+00 5.23e-01f  1
1854  5.8767099e+02 2.22e-04 9.51e-02  -3.8 1.49e+00  -1.2 1.00e+00 1.00e+00f  1
1855  5.8371150e+02 2.34e-03 9.47e-02  -3.8 4.45e+00  -1.7 1.00e+00 1.00e+00f  1
1856  5.7202371e+02 2.22e-02 1.53e+03  -3.8 1.32e+01  -2.1 1.60e-01 1.00e+00f  1
1857  5.7202837e+02 1.56e-02 2.93e+02  -3.8 6.51e-03   2.8 5.23e-01 3.40e-01h  1
1858  5.7202891e+02 1.45e-02 2.05e+04  -3.8 2.43e-03   3.2 8.84e-01 7.21e-02h  4
1859  5.7203596e+02 1.76e-03 1.51e+04  -3.8 3.18e-03   2.7 5.91e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1860  5.7203525e+02 2.57e-08 1.22e+04  -3.8 5.58e-04   3.2 6.67e-01 1.00e+00h  1
1861  5.7203026e+02 4.48e-03 9.25e+00  -3.8 4.93e-03   2.7 1.00e+00 1.00e+00f  1
1862  5.7201444e+02 1.38e-02 5.59e+03  -3.8 1.67e-02   2.2 1.91e-01 3.96e-01h  1
1863  5.7201330e+02 1.18e-02 1.70e+04  -3.8 4.81e-03   2.6 1.00e+00 1.52e-01f  2
1864  5.7199624e+02 5.63e-03

2022  9.2286910e+01 1.10e+01 1.40e+05  -3.8 2.44e+02    -  8.19e-02 1.00e+00f  1
2023  9.2309198e+01 1.04e+01 2.91e+04  -3.8 1.93e-01   0.7 9.56e-01 5.66e-02h  1
2024  9.2748177e+01 4.32e+00 9.46e+03  -3.8 2.28e-01   0.3 1.00e+00 5.88e-01h  1
2025  9.2648343e+01 4.34e+00 9.44e+03  -3.8 1.01e+05    -  7.64e-04 7.03e-04f  1
2026  9.1390829e+01 6.28e+00 1.15e+04  -3.8 1.30e+04    -  1.12e-03 4.57e-03f  1
2027  9.1443957e+01 5.52e+00 9.90e+03  -3.8 2.33e-01  -0.2 1.85e-04 1.54e-01h  1
2028  9.1518813e+01 3.60e+00 9.56e+03  -3.8 6.38e-02   1.1 1.00e+00 3.60e-01h  1
2029  9.1358952e+01 3.74e+00 9.87e+03  -3.8 1.06e+05    -  6.00e-04 1.80e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2030  9.1438984e+01 1.58e+00 2.58e+03  -3.8 4.43e-02   0.6 7.62e-01 6.05e-01h  1
2031  9.0506218e+01 1.92e+00 2.22e+03  -3.8 6.76e+02    -  7.45e-02 6.75e-02f  1
2032  9.0508209e+01 1.91e+00 2.10e+03  -3.8 3.73e+03    -  5.60e-02 6.43e-03h  1
2033  9.0497801e+01 1.85e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2120  8.1821478e+01 6.89e-05 3.21e+03  -5.7 3.10e+01    -  7.56e-01 5.40e-01f  1
2121  8.1807323e+01 1.03e-01 2.40e+03  -5.7 4.55e+01    -  1.45e-01 8.98e-01h  1
2122  8.1805037e+01 1.27e+00 8.82e+01  -5.7 2.59e-01  -1.5 1.00e+00 8.74e-01h  1
2123  8.1804503e+01 1.35e+00 8.59e+02  -5.7 9.31e-01  -2.0 3.23e-01 6.66e-02h  1
2124  8.1803485e+01 1.29e+00 2.21e+02  -5.7 1.14e+02    -  1.00e+00 2.29e-01h  1
2125  8.1803060e+01 1.05e+00 8.03e+01  -5.7 7.62e+01    -  1.00e+00 3.27e-01h  1
2126  8.1802834e+01 5.22e-03 2.35e-02  -5.7 4.33e+00    -  1.00e+00 1.00e+00h  1
2127  8.1802803e+01 9.14e-09 1.29e-03  -5.7 3.74e+00    -  1.00e+00 1.00e+00h  1
2128  8.1802802e+01 7.28e-11 4.37e-06  -5.7 2.60e-01    -  1.00e+00 1.00e+00h  1
2129  8.1802349e+01 7.77e-04 2.16e-01  -8.6 8.66e-01    -  9.95e-01 9.94e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2130  8.1802347e+01 2.91e-11

In [6]:
solve_pyomo(para_truth[2])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 274r 3.9384129e+04 8.98e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 5.24e-09R  2
 275r 3.9286485e+04 8.98e+04 9.99e+02   5.0 2.06e+08    -  1.22e-08 2.06e-07f  1
 276r 3.9395278e+04 8.98e+04 4.45e+03   2.9 2.53e+05    -  5.99e-03 1.83e-04f  1
 277r 4.0257286e+04 8.96e+04 4.36e+03   2.9 3.04e+05    -  6.35e-04 7.50e-04f  1
 278r 4.1444311e+04 8.93e+04 5.16e+03   2.9 3.48e+05    -  2.06e-03 8.99e-04f  1
 279r 4.6333675e+04 8.80e+04 3.58e+03   2.9 4.27e+05    -  7.20e-04 2.95e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280r 4.8608467e+04 8.74e+04 4.67e+03   2.9 5.22e+05    -  2.67e-03 1.12e-03f  1
 281r 5.0285247e+04 8.70e+04 9.79e+03   2.9 6.00e+05    -  5.83e-03 7.11e-04f  1
 282r 5.3390775e+04 8.62e+04 8.89e+03   2.9 7.06e+05    -  6.59e-04 1.09e-03f  1
 283r 5.4184942e+04 8.60e+04 8.88e+03   2.9 7.30e+05    -  2.71e-04 2.70e-04f  1
 284r 5.5126573e+04 8.58e+04 1.45e+04   2.9 5.32e+05    -  2.46e-03 4.36e-04f  1
 285r 5.7053463e+04 8.53e+04

 509  3.0351277e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -2.0 4.04e-05 2.17e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  3.0352011e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -1.6 2.07e-05 3.37e-05h  1
 511  3.0352439e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -2.1 3.09e-07 2.28e-05h  1
 512  3.0352685e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -1.6 2.95e-04 1.20e-05h  1
 513  3.0353288e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -2.1 6.52e-05 3.97e-05h  1
 514  3.0353841e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -1.7 3.71e-05 3.17e-05h  1
 515  3.0354282e+05 2.65e+04 7.99e+03  -1.0 2.65e+04  -2.2 7.04e-06 3.06e-05h  1
 516r 3.0354282e+05 2.65e+04 9.99e+02   4.4 0.00e+00  -1.7 0.00e+00 3.41e-07R  4
 517r 3.0354410e+05 2.65e+04 1.98e+04   4.4 1.29e+08    -  5.09e-05 3.86e-08f  1
 518r 3.1367303e+05 2.41e+04 3.26e+04   2.3 1.29e+06    -  5.29e-03 1.88e-03f  1
 519r 3.2721957e+05 2.08e+04 4.44e+04   2.3 1.45e+06    -  5.11e-03 2.22e-03f  1
iter    objective    inf_pr 

 604  4.0935202e+05 4.20e+02 6.15e+05  -1.0 2.51e+04  -3.6 3.50e-02 1.08e-02f  1
 605  4.0933476e+05 4.19e+02 6.11e+05  -1.0 2.64e+04  -4.1 4.29e-03 9.29e-04f  1
 606  4.0904415e+05 4.10e+02 5.44e+05  -1.0 2.53e+04  -3.6 1.06e-01 2.20e-02f  1
 607  4.0901982e+05 4.10e+02 5.43e+05  -1.0 2.69e+04  -4.1 8.52e-04 1.29e-03f  1
 608  4.0865651e+05 3.99e+02 5.28e+05  -1.0 2.53e+04  -3.7 2.81e-02 2.60e-02f  1
 609  4.0864662e+05 3.99e+02 5.28e+05  -1.0 3.78e+04  -4.2 2.37e-04 5.57e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  4.0863207e+05 3.98e+02 4.89e+05  -1.0 2.53e+04  -3.7 6.01e-02 1.01e-03f  1
 611  4.0853386e+05 3.97e+02 4.67e+05  -1.0 2.69e+04  -4.2 2.67e-02 4.15e-03f  1
 612  4.0827177e+05 3.90e+02 4.70e+05  -1.0 2.58e+04  -3.8 3.12e-04 1.66e-02f  1
 613  4.0794295e+05 3.85e+02 4.59e+05  -1.0 2.65e+04  -4.3 1.82e-02 1.22e-02f  1
 614  4.0748670e+05 3.75e+02 4.54e+05  -1.0 2.62e+04  -3.8 1.56e-02 2.60e-02f  1
 615  4.0709666e+05 3.70e+02

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 770  3.2622703e+05 1.31e+02 6.63e+03  -1.0 3.51e+02  -0.1 2.11e-03 3.76e-05h  1
 771  3.2622765e+05 1.31e+02 8.80e+03  -1.0 1.16e+03  -0.6 5.98e-04 3.09e-04h  1
 772  3.2622893e+05 1.31e+02 5.62e+03  -1.0 3.45e+03  -1.0 2.49e-04 6.24e-04h  1
 773  3.2623128e+05 1.31e+02 4.41e+03  -1.0 4.00e+03  -1.5 1.54e-04 6.79e-04h  1
 774  3.2623461e+05 1.31e+02 8.10e+03  -1.0 4.08e+03  -2.0 4.30e-03 8.80e-04h  1
 775  3.2632221e+05 1.28e+02 9.58e+03  -1.0 4.13e+03  -2.5 3.83e-04 2.25e-02h  1
 776  3.2635376e+05 1.27e+02 6.84e+03  -1.0 4.09e+03  -2.9 2.69e-02 8.47e-03h  1
 777  3.2675425e+05 1.12e+02 7.02e+03  -1.0 4.18e+03  -3.4 6.71e-02 1.19e-01h  1
 778  3.2677379e+05 1.11e+02 1.04e+04  -1.0 4.08e+03  -3.9 1.78e-01 1.04e-02h  1
 779  3.2655955e+05 9.98e+01 1.44e+04  -1.0 5.31e+03  -4.4 2.63e-01 9.76e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 780  3.2607404e+05 9.79e+01

...

1577  2.1070974e+04 1.82e+01 2.53e+03  -3.8 1.25e+03  -3.6 2.12e-03 2.02e-04f  1
1578  2.1070359e+04 1.82e+01 2.53e+03  -3.8 1.15e+04  -4.1 9.50e-05 7.48e-05f  1
1579  2.1063497e+04 1.82e+01 2.53e+03  -3.8 1.33e+03  -3.7 1.69e-03 1.61e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1580  2.1063493e+04 1.82e+01 2.53e+03  -3.8 1.60e+04    -  1.66e-04 2.95e-07f  1
1581  2.1055647e+04 1.82e+01 2.53e+03  -3.8 3.95e+03    -  2.47e-03 8.16e-04f  1
1582  2.1042015e+04 1.82e+01 2.52e+03  -3.8 4.62e+03    -  1.53e-03 1.35e-03f  1
1583  2.1028716e+04 1.81e+01 2.52e+03  -3.8 2.29e+03  -4.2 6.08e-03 2.07e-03f  1
1584  2.1027362e+04 1.81e+01 2.51e+03  -3.8 1.40e+03  -3.7 4.46e-03 3.12e-04f  1
1585  2.1024891e+04 1.81e+01 2.51e+03  -3.8 2.80e+04  -4.2 4.53e-04 2.88e-04f  1
1586  2.0959791e+04 1.79e+01 2.44e+03  -3.8 1.46e+03  -3.8 4.66e-02 1.44e-02f  1
1587  2.0908404e+04 1.75e+01 2.40e+03  -3.8 8.00e+02  -3.4 1.55e-02 2.09e-02f  1
1588  2.0746737e+04 1.69e+01

1681  1.1739323e+03 9.10e-02 7.12e+02  -3.8 3.93e+01  -2.1 1.83e-02 3.01e-03f  1
1682  1.1738129e+03 8.98e-02 6.72e+02  -3.8 1.75e+01  -0.8 4.08e-02 1.48e-02f  1
1683  1.1509947e+03 4.74e-02 2.26e+02  -3.8 5.41e+00  -1.3 7.97e-01 1.00e+00f  1
1684  1.1422609e+03 4.84e-02 1.23e+02  -3.8 1.64e+01  -1.8 4.32e-01 1.26e-01f  1
1685  1.1422607e+03 4.80e-02 2.38e+02  -3.8 8.69e-03   2.3 2.27e-01 6.98e-03h  1
1686  1.1422348e+03 7.82e-07 5.29e-01  -3.8 8.59e-03   1.8 1.00e+00 1.00e+00f  1
1687  1.1421960e+03 5.07e-07 4.22e+02  -3.8 1.59e-02   1.3 1.00e+00 5.19e-01f  1
1688  1.1420505e+03 7.31e-06 2.43e+02  -3.8 4.10e-02   0.8 1.00e+00 8.27e-01f  1
1689  1.1415741e+03 3.76e-05 1.29e+02  -3.8 1.14e-01   0.4 6.61e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1690  1.1401477e+03 7.99e-05 2.59e-01  -3.8 3.41e-01  -0.1 1.00e+00 1.00e+00f  1
1691  1.1374828e+03 3.11e-04 3.50e+00  -3.8 1.02e+00  -0.6 4.46e-01 6.23e-01f  1
1692  1.1341353e+03 1.18e-03

1861  1.6258493e+02 5.67e+00 1.18e+03  -3.8 9.14e+02    -  1.53e-02 6.33e-05h  1
1862  1.5932648e+02 5.66e+00 1.17e+03  -3.8 9.29e+02    -  2.46e-06 2.67e-03f  1
1863  1.5932087e+02 5.65e+00 1.24e+03  -3.8 9.14e+02    -  2.11e-02 7.27e-05h  1
1864  1.5629592e+02 5.64e+00 1.22e+03  -3.8 9.45e+02    -  3.39e-06 3.50e-03f  1
1865  1.5628954e+02 5.64e+00 1.30e+03  -3.8 9.15e+02    -  2.68e-02 8.46e-05h  1
1866  1.5349201e+02 5.62e+00 1.28e+03  -3.8 9.54e+02    -  4.28e-06 4.13e-03f  1
1867  1.5348464e+02 5.62e+00 1.38e+03  -3.8 9.16e+02    -  3.23e-02 9.97e-05h  1
1868  1.5090681e+02 5.60e+00 1.35e+03  -3.8 9.62e+02    -  5.07e-06 4.53e-03f  1
1869  1.5089806e+02 5.59e+00 1.37e+03  -3.8 9.16e+02    -  5.90e-03 1.19e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1870  1.5086261e+02 5.59e+00 1.36e+03  -3.8 6.12e+00  -1.3 3.84e-03 1.47e-03f  1
1871  1.4620929e+02 8.51e+00 1.45e+03  -3.8 4.25e+01  -1.8 1.26e-04 2.36e-02f  1
1872  1.4620727e+02 8.51e+00

2017  5.8159842e+01 1.44e-01 1.88e+03  -5.7 2.36e+02  -6.3 6.68e-01 4.86e-02f  1
2018  5.8159838e+01 1.44e-01 1.88e+03  -5.7 3.82e+02  -6.7 1.27e-03 1.11e-06h  2
2019  5.8159843e+01 1.44e-01 1.62e+03  -5.7 6.59e+03  -7.2 3.53e-03 9.60e-08f  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2020  5.8154788e+01 1.37e-01 1.49e+03  -5.7 4.80e+02  -7.7 8.57e-02 6.63e-02h  1
2021  5.8147512e+01 1.37e-01 1.32e+03  -5.7 1.34e+03  -8.2 1.22e-01 3.55e-02f  1
2022  5.8143744e+01 1.37e-01 1.28e+03  -5.7 2.88e+03  -8.6 5.88e-02 9.59e-03h  1
2023  5.8141491e+01 1.37e-01 1.28e+03  -5.7 5.38e+03  -7.3 1.13e-06 7.52e-05f  1
2024  5.8129831e+01 1.37e-01 1.43e+03  -5.7 1.32e+03  -6.9 3.57e-02 1.58e-03f  1
2025  5.8062563e+01 1.40e-01 1.36e+03  -5.7 9.71e+03  -7.4 4.92e-04 2.40e-03f  1
2026  5.7721384e+01 4.87e-01 1.23e+03  -5.7 4.95e+03  -7.8 5.93e-03 1.26e-02f  1
2027  5.7654617e+01 5.11e-01 1.21e+03  -5.7 1.21e+03  -8.3 1.61e-02 1.56e-02f  1
2028  5.7510687e+01 7.13e-01

In iteration 2202, 1 Slack too small, adjusting variable bound
2203  5.5223718e+01 8.89e-01 4.64e+01  -8.6 2.89e-01  -3.4 7.32e-01 7.98e-01h  1
2204  5.5223685e+01 2.97e-01 1.72e+01  -8.6 9.97e-02  -2.9 9.78e-01 8.61e-01h  1
2205  5.5223678e+01 3.87e-01 4.98e+02  -8.6 4.58e-01  -3.4 8.97e-01 1.14e-01h  1
2206  5.5223670e+01 1.84e-01 2.64e+02  -8.6 7.70e-02  -3.0 4.39e-01 5.67e-01h  1
2207  5.5223667e+01 3.18e-07 1.11e+01  -8.6 2.30e-02  -2.6 1.00e+00 8.06e-01h  1
2208  5.5223656e+01 1.13e+00 1.71e+02  -8.6 1.10e-01  -3.0 1.89e-01 1.00e+00h  1
2209  5.5223643e+01 2.48e-01 6.58e+00  -8.6 5.06e-02  -2.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2210  5.5223586e+01 1.03e+01 3.46e+02  -8.6 3.53e-01  -3.1 3.78e-01 7.51e-01h  1
2211  5.5223558e+01 7.26e+00 3.71e+02  -8.6 1.72e-01  -2.7 1.00e+00 3.18e-01h  1
2212  5.5223540e+01 7.12e+00 9.50e+02  -8.6 9.49e-01  -3.1 2.69e-01 4.24e-02h  1
2213  5.5223540e+01 7.12e+00 3.81e+02  -8.6 1.

In [7]:
solve_pyomo(para_truth[3])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 221r 7.5578411e+04 8.07e+04 9.99e+02   4.9 0.00e+00   2.6 0.00e+00 2.87e-07R  6
 222r 7.5571414e+04 8.07e+04 9.99e+02   4.9 3.36e+07    -  6.98e-07 1.85e-07f  1
 223r 7.5917925e+04 8.06e+04 7.42e+03   2.8 5.65e+05    -  8.70e-03 1.61e-04f  1
 224r 7.7127318e+04 8.03e+04 9.23e+03   2.8 7.85e+05    -  1.78e-03 3.89e-04f  1
 225r 7.8045157e+04 8.01e+04 1.19e+04   2.8 5.08e+05    -  1.10e-03 4.47e-04f  1
 226r 8.2950371e+04 7.89e+04 4.89e+03   2.8 5.11e+05    -  9.13e-04 2.35e-03f  1
 227r 8.9698159e+04 7.73e+04 5.42e+03   2.8 5.74e+05    -  1.03e-03 2.75e-03f  1
 228r 8.9728798e+04 7.73e+04 5.59e+03   2.8 7.49e+02   0.0 9.65e-03 9.87e-03f  1
 229r 8.9769569e+04 7.73e+04 1.63e+05   2.1 4.52e+01   1.3 9.92e-01 3.97e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 230r 8.9422403e+04 7.73e+04 7.31e+04   2.1 1.30e+02   0.9 5.25e-01 7.38e-01f  1
 231r 8.9405017e+04 7.73e+04 9.86e+04   2.1 2.11e+02   0.4 4.84e-01 2.25e-01f  1
 232r 8.9798203e+04 7.73e+04

 423  3.2274388e+05 1.82e+04 4.16e+04  -1.0 3.46e+04  -4.6 5.64e-03 3.10e-03h  1
 424  3.2288375e+05 1.82e+04 4.10e+04  -1.0 3.49e+04  -5.0 1.36e-02 3.30e-03h  1
 425  3.2289002e+05 1.82e+04 4.10e+04  -1.0 3.45e+04  -4.6 5.68e-03 1.39e-04h  1
 426  3.2395236e+05 1.77e+04 4.08e+04  -1.0 3.66e+04  -5.1 5.18e-03 2.79e-02h  1
 427  3.2395415e+05 1.77e+04 4.98e+04  -1.0 3.39e+04  -3.8 3.39e-02 3.92e-05h  1
 428  3.2457473e+05 1.74e+04 4.90e+04  -1.0 3.42e+04  -4.2 9.79e-03 1.37e-02h  1
 429  3.2458485e+05 1.74e+04 4.90e+04  -1.0 3.46e+04    -  3.14e-04 2.45e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430  3.2473002e+05 1.74e+04 4.88e+04  -1.0 3.40e+04    -  3.69e-03 3.38e-03h  1
 431  3.2473432e+05 1.73e+04 4.88e+04  -1.0 7.73e+04    -  4.82e-05 2.41e-03h  1
 432  3.2479719e+05 1.73e+04 4.83e+04  -1.0 3.46e+04    -  8.05e-03 2.03e-03h  1
 433  3.2479170e+05 1.73e+04 4.83e+04  -1.0 7.41e+04    -  2.93e-04 5.85e-04f  1
 434  3.2479183e+05 1.73e+04

 517  3.1887555e+05 8.95e+02 3.85e+03  -1.0 5.88e+04  -5.8 2.87e-02 2.93e-03f  1
 518  3.1788925e+05 8.15e+02 3.55e+03  -1.0 1.31e+04  -5.4 6.95e-02 8.95e-02f  1
 519  3.1769390e+05 7.84e+02 3.73e+03  -1.0 1.01e+04  -5.0 2.60e-02 3.81e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 520  3.1760041e+05 7.35e+02 5.58e+03  -1.0 9.78e+03  -4.5 6.16e-04 6.24e-02f  1
 521  3.1679040e+05 6.31e+02 4.70e+03  -1.0 9.33e+03  -5.0 1.48e-01 1.42e-01f  1
 522  3.1494666e+05 5.47e+02 4.02e+03  -1.0 1.78e+04  -5.5 1.02e-01 1.33e-01f  1
 523  3.1494686e+05 5.47e+02 3.97e+03  -1.0 5.70e+03  -4.2 4.78e-03 2.46e-04h  1
 524  3.1487541e+05 5.27e+02 3.25e+03  -1.0 7.28e+03  -4.6 6.23e-02 3.65e-02f  1
 525  3.1413303e+05 4.71e+02 2.92e+03  -1.0 8.25e+03  -5.1 1.05e-01 1.06e-01f  1
 526  3.1413323e+05 4.71e+02 3.04e+03  -1.0 5.83e+03  -3.8 1.96e-02 1.21e-04h  1
 527  3.1412414e+05 4.62e+02 2.84e+03  -1.0 6.31e+03  -4.3 2.43e-02 1.97e-02f  1
 528  3.1343026e+05 3.75e+02

 684  2.9471363e+05 2.09e-02 1.95e+03  -2.5 1.65e+01  -2.4 1.00e+00 1.74e-01f  1
 685  2.9470689e+05 1.84e-02 2.19e+03  -2.5 5.06e+01  -2.9 5.81e-01 1.27e-01f  1
 686  2.9469667e+05 1.72e-02 2.65e+03  -2.5 1.56e+02  -3.4 4.78e-01 6.64e-02f  1
 687  2.9467257e+05 1.61e-02 3.31e+03  -2.5 4.77e+02  -3.8 6.02e-01 5.30e-02f  1
 688  2.9460245e+05 8.03e-02 3.35e+03  -2.5 1.44e+03  -4.3 1.67e-01 5.14e-02f  1
 689  2.9414091e+05 2.89e+00 2.84e+03  -2.5 4.33e+03  -4.8 7.92e-02 1.10e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 690  2.9343994e+05 8.54e+00 3.12e+03  -2.5 1.28e+04  -5.3 1.06e-01 5.42e-02f  1
 691  2.9248147e+05 1.88e+01 2.59e+03  -2.5 4.12e+04  -5.8 1.35e-03 2.23e-02f  1
 692  2.9247407e+05 1.88e+01 2.75e+03  -2.5 1.47e+04  -5.3 1.96e-02 5.19e-04f  1
 693  2.9236670e+05 1.89e+01 2.67e+03  -2.5 5.86e+04  -5.8 3.75e-05 1.94e-03f  1
 694  2.9235747e+05 1.89e+01 3.72e+03  -2.5 1.64e+04  -5.4 1.08e-01 5.73e-04f  1
 695  2.9132587e+05 1.46e+02

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1150  2.7910584e+04 1.00e+02 7.86e+03  -3.8 6.60e+03  -4.1 1.67e-05 2.87e-03f  1
1151  2.7910494e+04 1.00e+02 7.74e+03  -3.8 9.52e+03  -4.6 1.19e-02 4.65e-06f  1
1152  2.7895724e+04 1.00e+02 7.74e+03  -3.8 2.30e+04  -5.1 7.60e-07 1.91e-04f  1
1153  2.6622234e+04 8.42e+01 8.03e+03  -3.8 3.03e+03  -3.8 7.18e-05 1.63e-01f  1
1154  2.6453606e+04 8.38e+01 7.76e+03  -3.8 9.22e+03  -4.2 1.79e-02 4.76e-03f  1
1155  2.6453579e+04 8.38e+01 7.76e+03  -3.8 1.36e+04  -4.7 7.70e-05 5.23e-07f  1
1156  2.6452110e+04 8.38e+01 7.74e+03  -3.8 2.40e+04  -5.2 1.13e-03 3.05e-05f  1
1157  2.6340540e+04 8.34e+01 7.59e+03  -3.8 1.40e+04  -4.8 1.49e-02 4.00e-03f  1
1158  2.5743241e+04 8.24e+01 7.69e+03  -3.8 2.09e+04  -5.2 2.16e-06 1.42e-02f  1
1159  2.5696223e+04 8.20e+01 7.44e+03  -3.8 3.95e+03  -3.9 2.67e-02 4.30e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1160  2.5695794e+04 8.20e+01

1251  6.8605140e+03 4.97e+00 1.36e+03  -3.8 4.32e+02  -3.1 2.14e-02 2.76e-02f  1
1252  6.8500140e+03 4.88e+00 1.44e+03  -3.8 2.02e+02  -2.7 1.04e-02 1.83e-02f  1
1253  6.8497855e+03 4.88e+00 1.42e+03  -3.8 6.47e+03    -  1.44e-03 2.76e-05f  1
1254  6.7925647e+03 4.84e+00 1.26e+03  -3.8 3.13e+03    -  4.59e-02 9.68e-03f  1
1255  6.4863992e+03 4.55e+00 1.15e+03  -3.8 2.15e+03    -  9.43e-02 5.97e-02f  1
1256  6.3461161e+03 4.42e+00 1.17e+03  -3.8 1.74e+03    -  9.40e-02 2.89e-02f  1
1257  5.7122096e+03 3.86e+00 9.34e+02  -3.8 1.73e+03    -  5.89e-03 1.31e-01f  1
1258  5.7110823e+03 3.85e+00 9.32e+02  -3.8 3.99e+03    -  4.02e-06 1.81e-04f  1
1259  5.6565468e+03 4.21e+00 1.01e+03  -3.8 3.80e+03    -  1.63e-02 8.16e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1260  5.6549685e+03 4.21e+00 1.08e+03  -3.8 1.55e+03    -  8.70e-02 3.62e-04f  1
1261  4.7954537e+03 3.95e+00 7.65e+02  -3.8 1.57e+03    -  6.82e-02 1.93e-01f  1
1262  4.5373400e+03 3.70e+00

1355  2.2684266e+02 1.40e+01 6.77e+03  -3.8 1.00e+02  -2.7 3.23e-02 3.01e-04f  1
1356  2.2474085e+02 1.40e+01 6.77e+03  -3.8 4.32e+04    -  2.74e-04 4.24e-04f  1
1357  2.2467981e+02 1.40e+01 6.79e+03  -3.8 2.59e+03  -3.2 3.76e-04 1.04e-05f  1
1358  2.2345521e+02 1.40e+01 6.78e+03  -3.8 1.52e+04    -  4.47e-06 4.89e-04f  1
1359  2.2345323e+02 1.40e+01 6.78e+03  -3.8 7.11e+01  -1.9 2.91e-03 9.36e-06h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1360  2.0844628e+02 1.43e+01 6.77e+03  -3.8 2.20e+04    -  8.33e-06 3.11e-03f  1
1361  2.0842331e+02 1.43e+01 6.77e+03  -3.8 9.16e+01  -2.4 3.41e-04 5.65e-05f  1
1362  2.0590060e+02 1.43e+01 6.78e+03  -3.8 1.10e+05    -  1.16e-06 8.12e-05f  1
1363  2.0587710e+02 1.43e+01 6.78e+03  -3.8 3.58e+04    -  1.97e-04 3.37e-06f  1
1364  2.0685276e+02 1.43e+01 6.73e+03  -3.8 4.19e+02  -2.8 2.00e-08 5.35e-04h  1
1365  2.0654354e+02 1.43e+01 6.73e+03  -3.8 5.67e+01  -1.5 1.65e-03 1.88e-03f  1
1366  2.0689415e+02 1.43e+01

1455  6.3693040e+01 7.77e+00 9.82e+02  -3.8 2.39e+01  -2.0 3.64e-02 2.27e-02f  1
1456  6.3293695e+01 6.91e+00 9.32e+02  -3.8 3.87e+00  -1.6 1.33e-01 1.08e-01h  1
1457  6.3102736e+01 6.85e+00 9.23e+02  -3.8 2.75e+01  -2.1 1.36e-02 7.29e-03f  1
1458  6.2897520e+01 6.51e+00 8.29e+02  -3.8 3.99e+00  -1.7 2.51e-02 4.90e-02h  1
1459  6.2534226e+01 6.46e+00 8.14e+02  -3.8 9.63e+01  -2.1 5.99e-03 4.49e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1460  6.2244140e+01 6.07e+00 1.08e+03  -3.8 4.95e+00  -1.7 2.16e-01 5.81e-02h  1
1461  6.1942699e+01 6.02e+00 1.07e+03  -3.8 5.92e+01  -2.2 7.79e-03 6.04e-03f  1
1462  6.1611511e+01 5.56e+00 1.18e+03  -3.8 5.68e+00  -1.8 1.69e-01 7.24e-02h  1
1463  6.1091198e+01 5.40e+00 1.15e+03  -3.8 3.25e+01  -2.3 1.62e-02 2.08e-02f  1
1464  6.0743681e+01 4.96e+00 1.09e+03  -3.8 6.18e+00  -1.8 8.68e-02 7.59e-02h  1
1465  6.0538262e+01 4.93e+00 1.08e+03  -3.8 4.71e+01  -2.3 1.30e-02 5.69e-03f  1
1466  6.0271868e+01 4.64e+00

1553  5.2225114e+01 1.15e-06 4.79e-01  -3.8 1.02e-01  -2.3 1.00e+00 1.00e+00h  1
1554  5.2224019e+01 4.70e-06 3.82e-01  -3.8 5.31e-01  -2.8 1.00e+00 1.00e+00H  1
1555  5.2224863e+01 1.91e-06 2.50e+01  -3.8 1.92e-01  -2.4 1.00e+00 5.00e-01h  2
1556  5.2225319e+01 1.35e-07 7.61e-02  -3.8 4.66e-02  -1.9 1.00e+00 1.00e+00h  1
1557  5.2225560e+01 7.15e-07 3.52e-01  -3.8 1.07e-01  -2.4 1.00e+00 1.00e+00h  1
1558  5.2225739e+01 2.50e+00 1.92e+01  -3.8 4.35e-01  -2.9 9.95e-01 5.00e-01h  2
1559  5.2225816e+01 1.67e+00 5.93e-01  -3.8 1.85e-01  -2.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1560  5.2225665e+01 2.40e-01 3.32e-01  -3.8 1.30e-01  -2.0 1.00e+00 1.00e+00h  1
1561  5.2223697e+01 7.49e-01 1.22e+01  -3.8 5.75e+02    -  3.62e-01 4.23e-01h  1
1562  5.2223885e+01 1.46e-02 4.34e+00  -3.8 9.35e-01  -2.5 1.00e+00 7.32e-01h  1
1563  5.2221308e+01 3.93e-01 4.17e+01  -3.8 4.00e+02    -  2.22e-01 1.00e+00h  1
1564  5.2221096e+01 3.40e-01

In [8]:
solve_pyomo(para_truth[4])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 187r 2.8235460e+04 9.30e+04 1.29e+04  -4.5 1.57e+02  -0.8 4.66e-03 6.42e-03f  1
 188r 2.8240533e+04 9.30e+04 1.29e+04  -4.5 5.51e+02  -1.3 7.25e-04 4.52e-03f  1
 189r 2.8240535e+04 9.30e+04 1.29e+04  -4.5 1.85e+04    -  2.56e-04 3.90e-08f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 2.8281647e+04 9.30e+04 1.35e+04  -4.5 8.08e+03    -  3.25e-02 1.15e-03f  1
 191r 2.8342927e+04 9.30e+04 1.45e+04  -4.5 7.98e+03    -  4.27e-02 1.71e-03f  1
 192r 2.8548857e+04 9.30e+04 2.58e+04  -4.5 7.78e+03    -  7.35e-02 5.75e-03f  1
 193r 2.8551431e+04 9.30e+04 2.97e+04  -4.5 7.56e+03    -  1.84e-02 7.24e-05f  1
 194r 2.8742117e+04 9.29e+04 2.84e+04  -4.5 7.56e+03    -  6.20e-07 5.39e-03f  1
 195r 2.8794781e+04 9.29e+04 3.32e+04  -4.5 7.47e+03    -  2.28e-02 1.53e-03f  1
 196r 2.8811994e+04 9.29e+04 3.31e+04  -4.5 7.46e+03    -  7.81e-07 4.99e-04f  1
 197r 2.8812000e+04 9.29e+04 3.33e+04  -4.5 7.59e+03    -  1.16e-03 2.40e-07f  1
 198r 2.8931629e+04 9.29e+04

 465  2.2661299e+05 1.07e+04 6.29e+03  -1.0 1.07e+04  -1.0 4.09e-04 1.44e-04h  1
 466r 2.2661299e+05 1.07e+04 9.99e+02   4.0 0.00e+00  -0.5 0.00e+00 4.87e-07R  3
 467r 2.2661241e+05 1.07e+04 1.72e+03   4.0 1.70e+07    -  4.06e-06 3.81e-08f  1
 468r 2.2739327e+05 1.04e+04 1.33e+05   2.6 2.29e+06    -  1.70e-02 1.47e-04f  1
 469r 2.5199923e+05 1.58e+02 1.17e+05   2.6 1.65e+06    -  2.60e-03 6.38e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 470  2.5200635e+05 1.58e+02 6.52e+03  -1.0 2.15e+02  -0.1 1.62e-03 8.96e-04h  1
 471  2.5200957e+05 1.58e+02 6.52e+03  -1.0 2.05e+02   0.3 6.33e-04 3.94e-04h  1
 472  2.5201673e+05 1.58e+02 6.51e+03  -1.0 2.19e+02  -0.2 6.43e-03 9.19e-04h  1
 473  2.5203522e+05 1.57e+02 6.50e+03  -1.0 2.08e+02   0.3 6.81e-03 2.29e-03h  1
 474  2.5204202e+05 1.57e+02 6.49e+03  -1.0 2.12e+02  -0.2 1.65e-02 8.74e-04h  1
 475  2.5204491e+05 1.57e+02 6.49e+03  -1.0 2.51e+02  -0.7 3.56e-03 3.92e-04h  1
 476  2.5204730e+05 1.57e+02

 649  2.5343704e+05 5.10e+00 6.39e+05  -1.0 1.80e+02  -2.2 3.32e-03 1.08e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650  2.5343666e+05 5.07e+00 6.36e+05  -1.0 8.23e+02  -2.7 4.06e-03 3.98e-03h  1
 651  2.5343615e+05 4.99e+00 6.17e+05  -1.0 4.19e+02  -2.3 2.13e-02 1.78e-02f  1
 652  2.5338448e+05 4.61e+00 9.74e+05  -1.0 6.48e+02  -2.8 8.74e-03 1.70e-01f  1
 653  2.5333985e+05 4.43e+00 5.19e+04  -1.0 2.65e+02  -3.2 5.00e-01 4.20e-02f  1
 654  2.5164323e+05 2.13e+00 5.11e+05  -1.0 7.44e+02  -3.7 1.08e-02 5.79e-01f  1
 655  2.5162843e+05 2.13e+00 5.04e+05  -1.0 2.16e+03  -4.2 9.56e-03 1.89e-03f  1
 656  2.5157671e+05 2.13e+00 4.77e+05  -1.0 7.21e+03  -4.7 4.08e-02 2.46e-03f  1
 657  2.5153900e+05 2.12e+00 4.24e+05  -1.0 2.24e+03  -4.3 7.25e-02 5.20e-03f  1
 658  2.5051005e+05 2.03e+00 5.13e+05  -1.0 9.16e+02  -3.8 1.95e-03 2.97e-01f  1
 659  2.4984853e+05 2.09e+00 5.02e+05  -1.0 2.88e+03  -4.3 2.93e-02 6.23e-02f  1
iter    objective    inf_pr 

 828  2.0673273e+05 6.67e-04 2.16e+02  -3.8 1.13e+02  -2.7 3.43e-02 9.65e-03f  1
 829  2.0670019e+05 2.85e-03 4.14e+02  -3.8 3.44e+02  -3.1 1.18e-01 1.98e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 830  2.0656745e+05 9.18e-02 6.70e+02  -3.8 1.04e+03  -3.6 7.66e-02 2.71e-02f  1
 831  2.0650114e+05 1.11e-01 8.61e+02  -3.8 3.45e+03  -4.1 2.46e-02 4.40e-03f  1
 832  2.0604906e+05 1.14e+00 1.58e+03  -3.8 1.17e+03  -3.7 2.44e-01 8.17e-02f  1
 833  2.0604307e+05 1.14e+00 1.57e+03  -3.8 3.66e+03  -4.1 2.11e-05 3.58e-04f  1
 834  2.0579958e+05 1.12e+00 2.49e+03  -3.8 1.32e+03  -3.7 4.68e-01 3.83e-02f  1
 835  2.0417218e+05 1.01e+01 1.31e+03  -3.8 4.45e+03  -4.2 6.94e-03 7.50e-02f  1
 836  2.0417175e+05 1.01e+01 1.67e+03  -3.8 1.53e+03  -3.8 9.86e-02 5.89e-05f  1
 837  2.0382279e+05 1.14e+01 1.28e+03  -3.8 5.70e+03  -4.2 2.84e-05 1.22e-02f  1
 838  2.0372114e+05 1.03e+01 1.14e+03  -3.8 2.11e+02  -2.9 6.81e-02 1.00e-01f  1
 839  2.0360272e+05 7.17e+00

...

1104  1.5007490e+05 2.77e+01 5.13e+03  -3.8 2.10e+01  -0.7 3.44e-02 3.79e-03h  1
1105  1.5007536e+05 2.74e+01 1.53e+04  -3.8 2.46e+01  -1.2 1.64e-01 8.68e-03h  1
1106  1.5008139e+05 2.05e+01 1.07e+04  -3.8 3.55e+01  -1.6 9.79e-02 2.53e-01h  1
1107  1.5008037e+05 2.02e+01 8.05e+03  -3.8 6.36e+01  -2.1 1.86e-01 1.29e-02f  1
1108  1.4999879e+05 1.53e+01 7.01e+03  -3.8 1.62e+02  -2.6 1.84e-01 2.43e-01f  1
1109  1.4975970e+05 1.22e+01 7.18e+03  -3.8 4.57e+02  -3.1 9.45e-02 2.13e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1110  1.4764338e+05 1.25e+01 4.57e+03  -3.8 1.36e+03  -3.5 4.57e-01 6.03e-01f  1
1111  1.4750745e+05 1.24e+01 3.91e+03  -3.8 4.19e+03  -4.0 2.25e-01 1.25e-02f  1
1112  1.4711433e+05 1.15e+01 1.80e+03  -3.8 1.55e+03  -3.6 1.00e+00 9.86e-02f  1
1113  1.3998015e+05 1.11e+02 1.38e+03  -3.8 4.75e+03  -4.1 2.86e-01 5.59e-01f  1
1114  1.3728527e+05 1.29e+02 1.42e+03  -3.8 1.87e+03  -3.6 1.00e+00 5.72e-01f  1
1115  1.3728497e+05 1.29e+02

1282  5.2218994e+03 6.11e+01 7.61e+03  -3.8 6.34e+03  -6.1 7.53e-06 4.31e-05f  1
1283  5.2180028e+03 6.11e+01 7.61e+03  -3.8 3.91e+04  -6.6 5.88e-05 5.40e-05f  1
1284  5.2177568e+03 6.11e+01 7.61e+03  -3.8 7.10e+03  -6.2 7.05e-06 3.20e-05f  1
1285  5.2143532e+03 6.11e+01 7.61e+03  -3.8 5.36e+04  -6.7 2.45e-05 3.45e-05f  1
1286  5.2143245e+03 6.11e+01 7.61e+03  -3.8 8.10e+03  -6.2 1.73e-05 3.59e-06f  1
1287  5.2143242e+03 6.11e+01 7.61e+03  -3.8 1.19e+05  -5.8 6.94e-09 9.66e-10f  2
1288  5.2143241e+03 6.11e+01 7.61e+03  -3.8 1.14e+04  -3.6 1.44e-10 5.05e-09f  2
1289  5.2143243e+03 6.11e+01 7.72e+03  -3.8 6.97e+01  -1.3 1.62e-04 6.47e-07f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1290  5.2142012e+03 6.11e+01 7.75e+03  -3.8 1.77e+02  -1.8 6.74e-05 1.72e-04f  1
1291  5.2115504e+03 6.09e+01 7.62e+03  -3.8 2.22e+02  -2.3 4.42e-03 3.08e-03f  1
1292  5.2085612e+03 6.07e+01 7.62e+03  -3.8 3.61e+02  -2.8 1.58e-03 2.30e-03f  1
1293  5.2033080e+03 6.06e+01

1462  6.8254928e+02 2.13e+00 2.76e+03  -3.8 1.61e+00  -0.4 3.39e-01 2.68e-01f  1
1463  6.8004644e+02 1.92e+00 2.63e+03  -3.8 4.82e+00  -0.8 2.19e-01 9.65e-02f  1
1464  6.7851862e+02 1.62e+00 2.19e+03  -3.8 1.81e+00  -0.4 1.38e-01 1.58e-01f  1
1465  6.7511707e+02 1.43e+00 2.24e+03  -3.8 5.41e+00  -0.9 4.42e-01 1.18e-01f  1
1466  6.7020368e+02 1.36e+00 2.16e+03  -3.8 1.62e+01  -1.4 1.21e-01 5.72e-02f  1
1467  6.6107885e+02 1.33e+00 2.07e+03  -3.8 4.85e+01  -1.8 2.46e-02 3.58e-02f  1
1468  6.5459314e+02 1.36e+00 2.05e+03  -3.8 1.47e+02  -2.3 6.79e-03 8.35e-03f  1
1469  6.5423216e+02 1.36e+00 2.05e+03  -3.8 5.44e+01  -1.9 1.42e-03 1.28e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1470  6.5388145e+02 1.36e+00 2.05e+03  -3.8 1.65e+02  -2.4 3.05e-04 4.12e-04f  1
1471  6.5387640e+02 1.36e+00 2.05e+03  -3.8 5.15e+02  -2.8 1.05e-06 1.90e-06f  1
1472  6.5387629e+02 1.36e+00 2.05e+03  -3.8 2.00e+03  -3.3 6.43e-09 1.05e-08f  2
1473  6.5386086e+02 1.36e+00

1647  1.1517372e+02 2.74e+00 2.89e+06  -3.8 1.92e+00   2.0 1.19e-01 3.69e-02h  2
1648  1.1526537e+02 2.53e+00 1.10e+06  -3.8 4.91e-01   2.4 1.00e+00 1.44e-01h  1
1649  1.1498010e+02 1.97e+00 5.32e+05  -3.8 4.35e+01    -  4.29e-01 2.28e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1650  1.1499996e+02 1.93e+00 4.89e+05  -3.8 3.14e-01   1.9 1.00e+00 1.84e-02h  1
1651  1.1302490e+02 5.61e-01 5.16e+05  -3.8 5.13e+01    -  4.12e-01 1.00e+00f  1
1652  1.1311769e+02 3.36e-02 2.11e+03  -3.8 2.94e-02   1.4 1.00e+00 9.41e-01h  1
1653  1.0757669e+02 2.96e+00 2.52e+03  -3.8 3.29e+02    -  4.10e-01 2.96e-01f  1
1654  1.0146152e+02 6.72e+00 3.40e+03  -3.8 4.99e+02    -  6.82e-02 2.89e-01f  1
1655  1.0158514e+02 5.58e+00 4.46e+04  -3.8 2.11e-01   1.0 1.00e+00 1.70e-01h  1
1656  1.0150113e+02 5.24e+00 9.80e+03  -3.8 4.70e+01    -  5.38e-01 6.18e-02h  1
1657  9.9269759e+01 2.65e+00 2.67e+04  -3.8 1.69e+02    -  1.97e-04 4.99e-01f  1
1658  9.9351237e+01 1.65e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1750  8.4130416e+01 1.04e-04 5.36e+01  -3.8 1.79e+01    -  4.80e-01 1.00e+00h  1
1751  8.4130385e+01 1.69e-08 3.09e-02  -3.8 1.44e-02  -1.4 1.00e+00 1.00e+00h  1
1752  8.4130364e+01 3.82e-09 8.05e-03  -3.8 7.05e-03  -0.9 1.00e+00 1.00e+00h  1
1753  8.4130256e+01 1.05e-07 2.04e-01  -3.8 4.21e-02  -1.4 1.00e+00 1.00e+00h  1
1754  8.4130126e+01 2.26e-08 5.26e-02  -3.8 2.00e-02  -1.0 1.00e+00 1.00e+00h  1
1755  8.4128903e+01 6.06e-07 5.80e+00  -3.8 1.50e-01  -1.5 1.00e+00 1.00e+00H  1
1756  8.4128526e+01 2.53e-07 1.87e-01  -3.8 4.67e-02  -1.0 1.00e+00 1.00e+00h  1
1757  8.4126739e+01 7.54e-08 4.56e+00  -3.8 1.65e-01  -1.5 1.00e+00 1.00e+00H  1
1758  8.4126647e+01 2.23e-03 1.70e+00  -3.8 6.43e-02  -1.1 1.00e+00 1.00e+00h  1
1759  8.4126381e+01 1.53e-02 6.19e+01  -3.8 2.19e-01  -1.6 1.00e+00 1.16e-01h  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1760  8.4121802e+01 4.32e-06

In [9]:
solve_pyomo(para_truth[5])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 184r 3.0369741e+04 9.25e+04 2.00e+04  -6.7 2.41e+02  -0.7 5.62e-02 3.68e-06f  1
 185r 3.0488438e+04 9.24e+04 1.88e+04  -6.7 7.11e+02  -1.2 1.05e-01 6.02e-02f  1
 186r 3.0488438e+04 9.24e+04 1.88e+04  -6.7 2.69e+02  -0.8 1.33e-01 1.27e-07f  1
 187r 3.0616819e+04 9.24e+04 1.79e+04  -6.7 7.89e+02  -1.3 2.22e-08 4.69e-02f  1
 188r 3.0786081e+04 9.24e+04 1.49e+04  -6.7 3.01e+02  -0.8 2.40e-03 1.66e-01f  1
 189r 3.0803861e+04 9.24e+04 1.48e+04  -6.7 8.98e+02  -1.3 1.29e-05 7.37e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 3.0875563e+04 9.23e+04 1.37e+04  -6.7 3.42e+02  -0.9 4.75e-01 7.70e-02f  1
 191r 3.0875563e+04 9.23e+04 1.37e+04  -6.7 9.86e+02  -1.4 1.61e-07 5.69e-08f  1
 192r 3.1001368e+04 9.23e+04 1.23e+04  -6.7 3.78e+02  -0.9 9.17e-02 9.57e-02f  1
 193r 3.1001368e+04 9.23e+04 1.23e+04  -6.7 1.43e+02  -0.5 1.03e-01 1.44e-07f  1
 194r 3.1159828e+04 9.22e+04 1.41e+04  -6.7 4.23e+02  -1.0 2.94e-01 1.05e-01f  1
 195r 3.1159828e+04 9.22e+04

 439r 1.3112001e+05 5.54e+04 9.99e+02   4.7 0.00e+00   2.1 0.00e+00 4.22e-07R  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440r 1.3098992e+05 5.55e+04 7.24e+03   4.7 3.41e+07    -  1.48e-05 1.23e-06f  1
 441r 1.3296227e+05 5.47e+04 9.68e+04   2.6 1.30e+06    -  1.61e-02 5.70e-04f  1
 442r 1.7844158e+05 3.58e+04 8.87e+04   2.6 3.04e+06    -  7.47e-03 6.39e-03f  1
 443  1.7845512e+05 3.58e+04 4.62e+03  -1.0 3.58e+04   1.6 1.40e-05 3.53e-05h  1
 444  1.7846848e+05 3.58e+04 4.65e+03  -1.0 3.58e+04   1.1 4.88e-05 3.53e-05h  1
 445  1.7848417e+05 3.58e+04 4.79e+03  -1.0 3.58e+04   0.7 9.92e-05 4.28e-05h  1
 446r 1.7848417e+05 3.58e+04 9.99e+02   4.6 0.00e+00   0.2 0.00e+00 4.17e-07R  4
 447r 1.7843564e+05 3.58e+04 2.36e+04   4.6 2.04e+07    -  1.14e-04 6.97e-07f  1
 448r 1.8042199e+05 3.50e+04 7.75e+04   2.5 2.46e+06    -  6.28e-03 3.24e-04f  1
 449r 2.1599205e+05 2.03e+04 6.57e+04   2.5 3.29e+06    -  4.14e-03 4.57e-03f  1
iter    objective    inf_pr 

 532  2.6305011e+05 3.52e+02 3.04e+06  -1.0 2.22e+03  -1.8 7.94e-03 6.75e-03h  1
 533  2.6305362e+05 3.52e+02 3.07e+06  -1.0 1.85e+03  -1.4 5.89e-03 7.85e-04h  1
 534  2.6307714e+05 3.50e+02 3.05e+06  -1.0 2.41e+03  -1.9 4.35e-03 5.72e-03h  1
 535  2.6308128e+05 3.49e+02 3.41e+06  -1.0 1.99e+03  -1.4 6.10e-02 9.83e-04h  1
 536  2.6312254e+05 3.46e+02 3.37e+06  -1.0 2.85e+03  -1.9 1.38e-02 1.16e-02h  1
 537  2.6316187e+05 3.42e+02 3.97e+06  -1.0 2.12e+03  -1.5 1.09e-01 1.01e-02h  1
 538  2.6316683e+05 3.42e+02 3.96e+06  -1.0 2.52e+04  -2.0 2.87e-03 2.29e-03h  1
 539  2.6317128e+05 3.41e+02 4.03e+06  -1.0 2.15e+03  -1.5 1.65e-02 1.20e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540  2.6319994e+05 3.39e+02 3.94e+06  -1.0 3.20e+03  -2.0 3.99e-02 8.58e-03h  1
 541  2.6320250e+05 3.38e+02 3.98e+06  -1.0 2.11e+03  -1.6 1.10e-02 6.92e-04h  1
 542  2.6330810e+05 3.29e+02 3.70e+06  -1.0 2.11e+03  -2.1 1.31e-01 2.93e-02h  1
 543  2.6332006e+05 3.28e+02

 699  2.3537501e+05 5.23e+00 1.07e+04  -1.0 1.38e+03  -3.8 1.81e-01 4.18e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 700  2.3519290e+05 5.18e+00 1.02e+04  -1.0 3.89e+03  -4.3 3.53e-02 8.63e-03f  1
 701  2.3448370e+05 5.38e+00 1.00e+04  -1.0 1.55e+04  -4.8 1.40e-02 1.30e-02f  1
 702  2.3303413e+05 8.45e+00 1.03e+04  -1.0 4.74e+05  -5.3 1.12e-04 1.06e-03f  1
 703  2.3098042e+05 2.40e+01 1.10e+04  -1.0 1.81e+04  -4.8 5.64e-03 2.70e-02f  1
 704  2.3026117e+05 2.79e+01 1.09e+04  -1.0 5.92e+03  -4.4 9.36e-03 2.23e-02f  1
 705  2.3022710e+05 2.79e+01 9.63e+03  -1.0 4.32e+04  -4.9 9.44e-03 1.90e-04f  1
 706  2.2932769e+05 3.66e+01 1.01e+04  -1.0 7.63e+03  -4.5 7.73e-04 2.18e-02f  1
 707  2.2931961e+05 3.66e+01 9.73e+03  -1.0 2.65e+03  -4.0 2.09e-02 5.69e-04f  1
 708  2.2931105e+05 3.65e+01 9.43e+03  -1.0 8.82e+02  -3.6 2.88e-02 1.84e-03f  1
 709  2.2931576e+05 3.34e+01 1.86e+04  -1.0 5.37e+00   0.4 1.11e-02 8.44e-02h  1
iter    objective    inf_pr 

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1020  9.0138938e+04 1.61e+01 7.81e+03  -2.5 1.43e+03  -3.4 2.98e-02 5.93e-02f  1
1021  8.9932792e+04 1.58e+01 7.93e+03  -2.5 4.24e+03  -3.9 2.92e-02 1.88e-02f  1
1022  8.9789510e+04 1.52e+01 9.13e+03  -2.5 1.58e+03  -3.4 1.06e-01 3.56e-02f  1
1023  8.9610354e+04 1.50e+01 1.02e+04  -2.5 4.73e+03  -3.9 6.26e-02 1.45e-02f  1
1024  8.9106719e+04 1.33e+01 7.86e+03  -2.5 1.75e+03  -3.5 4.25e-02 1.11e-01f  1
1025  8.8708696e+04 1.30e+01 7.83e+03  -2.5 5.22e+03  -4.0 3.67e-02 2.87e-02f  1
1026  8.8581437e+04 1.26e+01 1.81e+04  -2.5 1.94e+03  -3.5 6.00e-01 2.49e-02f  1
1027  8.8289544e+04 1.24e+01 1.99e+04  -2.5 5.79e+03  -4.0 2.27e-01 1.87e-02f  1
1028  8.7308636e+04 1.03e+01 1.60e+04  -2.5 2.13e+03  -3.6 2.95e-03 1.71e-01f  1
1029  8.4878598e+04 1.21e+01 1.40e+04  -2.5 6.48e+03  -4.1 3.81e-01 1.36e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1030  8.4862272e+04 1.21e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1190  5.6061784e+03 5.82e-04 1.43e+04  -2.5 7.46e-01   0.3 6.70e-01 4.56e-01f  1
1191  5.6049613e+03 4.32e-04 1.70e+03  -2.5 1.60e+00  -0.1 5.55e-01 2.39e-01f  1
1192  5.6049638e+03 1.43e-07 1.62e+00  -2.5 1.59e-03   3.0 1.00e+00 1.00e+00f  1
1193  5.6049574e+03 4.01e-08 3.84e+02  -2.5 4.22e-03   2.5 1.00e+00 9.18e-01f  1
1194  5.6049477e+03 2.84e-07 1.46e+00  -2.5 1.29e-02   2.1 1.00e+00 1.00e+00f  1
1195  5.6049447e+03 4.65e-07 1.46e+00  -2.5 4.84e-03   2.5 1.00e+00 1.00e+00f  1
1196  5.6049239e+03 1.19e-07 1.45e+00  -2.5 1.44e-02   2.0 1.00e+00 1.00e+00f  1
1197  5.6048621e+03 1.91e-06 2.05e+03  -2.5 4.25e-02   1.5 1.00e+00 6.83e-01f  1
1198  5.6048587e+03 3.00e-07 1.42e+00  -2.5 1.99e-03   2.9 1.00e+00 1.00e+00f  1
1199  5.6048471e+03 2.01e-08 1.41e+00  -2.5 5.92e-03   2.4 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1200  5.6048071e+03 8.16e-08

1368  4.5126185e+02 4.16e-06 6.01e-01  -2.5 2.19e-02   1.4 1.00e+00 1.00e+00f  1
1369  4.5109209e+02 4.20e-06 1.13e+05  -3.8 4.31e-02   1.0 5.77e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1370  4.5088062e+02 6.16e-05 2.94e+00  -3.8 6.34e-02   0.5 1.00e+00 1.00e+00f  1
1371  4.5077871e+02 1.48e-04 7.42e+02  -3.8 1.02e-01   0.0 3.60e-01 5.40e-01f  1
1372  4.5065119e+02 4.66e-04 1.87e+03  -3.8 3.20e-01  -0.5 1.00e+00 2.86e-01f  1
1373  4.5059765e+02 5.27e-04 1.60e+03  -3.8 6.32e-01  -0.9 2.75e-01 5.94e-02f  1
1374  4.5046718e+02 1.96e-03 7.41e+02  -3.8 1.22e+00  -1.4 5.28e-01 1.48e-01f  1
1375  4.5019279e+02 5.80e-03 6.82e+02  -3.8 4.04e+00  -1.9 9.82e-02 7.38e-02f  1
1376  4.4988297e+02 7.89e-03 6.70e+02  -3.8 1.10e+01  -2.4 2.50e-02 3.67e-02f  1
1377  4.4983950e+02 7.69e-03 4.85e+02  -3.8 9.06e+01  -2.9 1.49e-02 8.34e-04f  1
1378  4.4951326e+02 3.08e-03 4.80e+02  -3.8 1.50e+02  -3.3 4.03e-03 3.91e-03f  1
1379  4.4636261e+02 3.13e-03

1467  1.9972486e+02 4.77e-07 4.19e+01  -3.8 3.16e-03   1.2 9.95e-01 1.00e+00h  1
1468  1.9972459e+02 3.41e-05 7.85e+01  -3.8 5.60e-02   0.7 4.54e-01 5.15e-01f  1
1469  1.9972381e+02 8.16e-06 3.22e-02  -3.8 1.78e-02   0.3 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1470  1.9971026e+02 9.82e-06 2.44e-02  -3.8 4.06e-02  -0.2 1.00e+00 1.00e+00f  1
1471  1.9967887e+02 2.35e-06 4.37e+00  -3.8 1.06e-01  -0.7 1.00e+00 8.89e-01f  1
1472  1.9957763e+02 7.53e-06 2.12e-02  -3.8 3.17e-01  -1.2 1.00e+00 1.00e+00f  1
1473  1.9927390e+02 1.37e-04 2.11e-02  -3.8 9.49e-01  -1.7 1.00e+00 1.00e+00f  1
1474  1.9836817e+02 1.54e-03 2.11e-02  -3.8 2.84e+00  -2.1 1.00e+00 1.00e+00f  1
1475  1.9677346e+02 4.25e-03 2.40e+01  -3.8 8.45e+00  -2.6 9.08e-01 5.95e-01f  1
1476  1.9677382e+02 3.66e-03 9.47e+02  -3.8 1.61e-02   2.3 3.21e-01 2.29e-01h  2
1477  1.9677381e+02 3.66e-03 7.48e+02  -3.8 5.18e-02   1.9 2.64e-02 7.23e-03h  3
1478  1.9677802e+02 1.32e-07

1564  7.2655603e+01 7.90e-01 6.08e+02  -5.7 1.99e+03    -  1.19e-02 3.37e-02f  1
1565  7.1916971e+01 2.25e+01 1.21e+02  -5.7 7.27e+02    -  3.80e-01 3.31e-01f  1
1566  7.1921664e+01 2.15e+01 1.14e+03  -5.7 1.14e-01   0.2 3.87e-01 4.27e-02h  1
1567  7.1839580e+01 2.06e+01 9.33e+02  -5.7 7.44e+02    -  5.45e-03 6.46e-02h  1
1568  7.1839582e+01 2.06e+01 4.65e+03  -5.7 2.77e+00   0.6 1.50e-01 1.77e-05h  1
1569  7.1836963e+01 2.06e+01 4.63e+03  -5.7 8.92e+03    -  2.26e-07 2.63e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1570  7.1838667e+01 1.99e+01 3.22e+03  -5.7 1.98e-01   0.1 2.61e-01 3.38e-02h  1
1571  7.1838579e+01 1.94e+01 3.21e+03  -5.7 2.28e+03    -  1.82e-04 2.85e-02h  1
1572  7.1838751e+01 1.93e+01 2.86e+03  -5.7 1.94e-01  -0.3 9.35e-01 3.76e-03h  1
1573  7.1836051e+01 1.91e+01 2.83e+03  -5.7 5.90e+03    -  7.65e-03 1.07e-02h  1
1574  7.1836054e+01 1.91e+01 2.92e+03  -5.7 1.81e-01  -0.8 1.00e+00 4.62e-05h  1
1575  7.1834381e+01 1.88e+01

In [10]:
solve_pyomo(para_truth[6])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 183r 1.7827848e+04 9.55e+04 1.21e+04  -6.7 2.76e+02  -0.8 4.37e-02 4.33e-01f  1
 184r 1.7920059e+04 9.55e+04 1.16e+04  -6.7 8.26e+02  -1.3 5.64e-02 3.79e-02f  1
 185r 1.8055090e+04 9.54e+04 9.79e+03  -6.7 3.10e+02  -0.9 1.72e-03 1.55e-01f  1
 186r 1.8058164e+04 9.54e+04 9.78e+03  -6.7 9.27e+02  -1.3 1.22e-01 1.20e-03f  1
 187r 1.8304144e+04 9.53e+04 7.30e+03  -6.7 3.48e+02  -0.9 1.31e-01 2.51e-01f  1
 188r 1.8304150e+04 9.53e+04 7.30e+03  -6.7 1.04e+03  -1.4 3.20e-02 2.06e-06f  1
 189r 1.8748888e+04 9.52e+04 4.43e+03  -6.7 3.92e+02  -1.0 7.95e-04 3.87e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.8748894e+04 9.52e+04 4.43e+03  -6.7 1.17e+03  -1.4 9.73e-02 1.70e-06f  1
 191r 1.9149014e+04 9.50e+04 3.04e+03  -6.7 4.40e+02  -1.0 2.35e-01 3.11e-01f  1
 192r 1.9149028e+04 9.50e+04 3.04e+03  -6.7 1.32e+03  -1.5 2.17e-05 3.80e-06f  1
 193r 1.9358441e+04 9.50e+04 2.60e+03  -6.7 4.95e+02  -1.1 2.00e-01 1.45e-01f  1
 194r 1.9358442e+04 9.50e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430  1.5068245e+05 1.34e+02 1.54e+05  -1.0 5.50e+02  -1.5 1.69e-01 2.12e-02h  1
 431  1.5080186e+05 1.28e+02 1.51e+05  -1.0 6.29e+02  -1.9 8.67e-02 4.59e-02h  1
 432  1.5085277e+05 1.26e+02 1.42e+05  -1.0 5.66e+02  -1.5 6.20e-03 1.98e-02h  1
 433  1.5085500e+05 1.26e+02 1.49e+05  -1.0 4.68e+02  -0.2 3.07e-03 8.16e-04h  1
 434  1.5085561e+05 1.26e+02 2.38e+05  -1.0 4.77e+02  -0.7 4.07e-02 2.33e-04h  1
 435  1.5091804e+05 1.22e+02 3.87e+05  -1.0 5.42e+02  -1.1 1.64e-01 2.43e-02h  1
 436  1.5110485e+05 1.13e+02 3.57e+05  -1.0 5.51e+02  -1.6 7.78e-02 7.72e-02h  1
 437  1.5122450e+05 1.07e+02 2.77e+05  -1.0 4.77e+02  -1.2 7.01e-04 5.27e-02h  1
 438  1.5123540e+05 1.06e+02 2.77e+05  -1.0 4.67e+02  -1.7 2.67e-02 5.35e-03h  1
 439  1.5130278e+05 1.03e+02 2.67e+05  -1.0 4.55e+02  -1.2 3.14e-02 3.21e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440  1.5150292e+05 9.22e+01

 619  2.5085785e+04 9.22e+01 3.88e+04  -1.0 1.12e+04    -  3.36e-02 8.56e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  2.5076526e+04 9.22e+01 3.75e+04  -1.0 5.42e+03    -  2.76e-02 4.82e-04f  1
 621  2.4214134e+04 8.93e+01 3.78e+04  -1.0 6.75e+03    -  3.62e-04 3.22e-02f  1
 622  2.4193874e+04 8.93e+01 3.69e+04  -1.0 8.44e+03    -  9.53e-03 6.00e-04f  1
 623  2.4076309e+04 8.87e+01 3.68e+04  -1.0 6.54e+03    -  5.35e-03 5.97e-03f  1
 624  2.2873844e+04 8.65e+01 3.78e+04  -1.0 9.87e+03    -  2.60e-04 2.71e-02f  1
 625  2.2860327e+04 8.65e+01 3.25e+04  -1.0 8.35e+03    -  1.06e-01 3.65e-04f  1
 626  2.2878527e+04 8.14e+01 3.08e+04  -1.0 1.28e+02  -1.0 7.05e-02 5.86e-02h  1
 627  2.2926640e+04 6.45e+01 3.03e+04  -1.0 1.39e+02  -1.5 1.39e-02 2.08e-01h  1
 628  2.2926819e+04 5.86e+01 2.02e+04  -1.0 1.69e+02  -2.0 2.58e-01 9.11e-02h  1
 629  2.2894208e+04 5.46e+01 1.09e+04  -1.0 3.05e+02  -2.5 3.11e-01 6.76e-02f  1
iter    objective    inf_pr 

 713  1.5765816e+03 8.70e+00 1.14e+04  -1.0 1.61e+04    -  2.21e-02 7.01e-04f  1
 714  1.5019422e+03 8.92e+00 1.13e+04  -1.0 8.56e+03    -  1.67e-03 3.86e-03f  1
 715  1.4999979e+03 7.31e+00 1.22e+04  -1.0 1.03e+00   0.1 2.71e-01 1.79e-01f  1
 716  1.4950914e+03 4.50e+00 9.80e+03  -1.0 1.86e+00  -0.4 4.40e-01 3.81e-01f  1
 717  1.3731816e+03 5.08e+00 9.81e+03  -1.0 7.53e+03    -  6.70e-03 5.53e-03f  1
 718  1.3720457e+03 4.45e+00 1.82e+04  -1.0 8.01e+00   0.0 2.78e-01 1.23e-01f  1
 719  1.3711834e+03 3.36e+00 6.73e+04  -1.0 4.09e-01   0.5 1.00e+00 2.41e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 720  1.1408506e+03 6.71e+00 6.65e+04  -1.0 1.01e+04    -  2.24e-03 6.28e-03f  1
 721  1.1402444e+03 4.78e+00 1.12e+05  -1.0 1.63e-01   1.8 9.75e-01 2.84e-01f  1
 722  1.1391590e+03 4.37e-03 2.39e+03  -1.0 1.63e-01   1.3 1.00e+00 9.90e-01f  1
 723  1.1392146e+03 2.21e-03 2.77e+06  -1.0 2.37e-02   1.7 1.00e+00 4.97e-01h  2
 724  1.1393027e+03 1.71e-04

 881  3.0357962e+02 2.44e-03 8.51e+01  -3.8 2.79e+01  -2.3 1.40e-01 2.88e-02f  1
 882  3.0005033e+02 2.08e-03 1.01e+02  -3.8 8.07e+01  -2.8 6.38e-02 4.34e-02f  1
 883  3.0001508e+02 2.07e-03 1.00e+02  -3.8 1.92e+02  -3.3 6.98e-03 3.82e-04f  1
 884  2.9698376e+02 2.06e-03 1.01e+02  -3.8 5.45e+02  -3.7 5.50e-05 4.85e-03f  1
 885  2.9694606e+02 2.06e-03 9.87e+01  -3.8 6.56e+02  -4.2 1.06e-02 1.24e-04f  1
 886  2.9450435e+02 2.06e-03 9.87e+01  -3.8 2.03e+03  -4.7 3.80e-06 9.20e-04f  1
 887  2.9444804e+02 2.06e-03 1.44e+02  -3.8 9.13e+02  -5.2 1.33e-02 1.34e-04f  1
 888  2.9218115e+02 2.06e-03 1.40e+02  -3.8 2.52e+03  -5.6 2.43e-06 7.80e-04f  1
 889  2.9210707e+02 2.06e-03 1.69e+02  -3.8 1.31e+03  -6.1 4.54e-03 1.62e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 890  2.9010137e+02 3.00e-03 2.21e+02  -3.8 2.26e+03  -6.6 1.17e-02 4.17e-03f  1
 891  2.8794034e+02 3.28e-03 2.15e+02  -3.8 1.42e+04  -7.1 2.10e-06 5.68e-04f  1
 892  2.8783750e+02 3.29e-03

1064  5.5986729e+01 1.69e+01 1.69e+02  -3.8 3.89e-01  -2.0 4.79e-01 1.03e-02h  1
1065  5.5985484e+01 1.67e+01 1.32e+02  -3.8 9.24e+01    -  1.91e-01 7.31e-03h  1
1066  5.5987272e+01 1.64e+01 1.14e+02  -3.8 1.68e-01  -2.5 5.15e-01 1.83e-02h  1
1067  5.5987752e+01 1.63e+01 1.02e+02  -3.8 3.39e-01  -3.0 1.00e+00 5.27e-03h  1
1068  5.5775258e+01 2.47e+00 1.82e+02  -3.8 1.19e+02    -  7.54e-03 8.51e-01h  1
1069  5.5777517e+01 1.85e+00 1.50e+01  -3.8 2.08e-02   0.2 1.00e+00 2.50e-01h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1070  5.5784149e+01 9.28e-07 3.10e-02  -3.8 7.67e-03   0.6 1.00e+00 1.00e+00h  1
1071  5.5781715e+01 7.79e-06 3.24e+00  -3.8 3.13e-02   0.1 1.00e+00 7.48e-01h  1
1072  5.5780875e+01 6.47e-08 1.74e-02  -3.8 4.82e-03   0.6 1.00e+00 1.00e+00f  1
1073  5.5780148e+01 2.27e-06 5.46e-02  -3.8 2.16e-02   0.1 1.00e+00 1.00e+00h  1
1074  5.5779314e+01 4.50e-07 2.98e-02  -3.8 9.29e-03   0.5 1.00e+00 1.00e+00h  1
1075  5.5778856e+01 5.77e-07

1162  5.4153756e+01 2.57e-07 3.61e-02  -3.8 3.30e-02  -1.7 1.00e+00 1.00e+00h  1
1163  5.4153760e+01 5.43e-07 1.74e-01  -3.8 6.92e-02  -2.2 1.00e+00 1.00e+00h  1
1164  5.4153745e+01 2.22e-07 6.28e-02  -3.8 4.93e-02  -2.6 1.00e+00 1.00e+00h  1
1165  5.4153742e+01 1.54e-08 9.93e-04  -3.8 1.50e-02  -2.2 1.00e+00 1.00e+00h  1
1166  5.4153737e+01 6.48e-08 8.37e-03  -3.8 4.00e-02  -2.7 1.00e+00 1.00e+00h  1
1167  5.4153735e+01 5.15e-09 1.13e-03  -3.8 1.46e-02  -2.3 1.00e+00 1.00e+00h  1
1168  5.4153730e+01 2.51e-07 3.11e-02  -3.8 4.58e-02  -2.7 1.00e+00 1.00e+00h  1
1169  5.4153724e+01 7.21e-08 7.72e-03  -3.8 2.28e-02  -2.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1170  5.4153697e+01 2.37e-06 5.13e-01  -3.8 2.74e-01  -2.8 1.00e+00 5.00e-01h  2
1171  5.4153641e+01 2.07e-06 2.39e-01  -3.8 1.10e-01  -2.4 1.00e+00 1.00e+00h  1
1172  5.4153588e+01 7.23e-07 7.55e-02  -3.8 5.76e-02  -1.9 1.00e+00 1.00e+00h  1
1173  5.4153484e+01 7.06e-01

In [11]:
solve_pyomo(para_truth[7])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 274r 4.1076663e+04 9.00e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 5.38e-12R  2
 275r 4.1076014e+04 9.00e+04 9.99e+02   5.0 1.56e+09    -  4.23e-09 1.85e-10f  1
 276r 4.0936189e+04 9.00e+04 9.99e+02   2.9 3.23e+06    -  3.66e-06 2.35e-05f  1
 277r 4.0953452e+04 9.00e+04 9.99e+02   2.9 3.49e+05    -  6.62e-04 2.13e-05f  1
 278r 4.1864116e+04 8.98e+04 1.46e+03   2.9 2.37e+05    -  1.77e-03 9.86e-04f  1
 279r 4.2745105e+04 8.95e+04 3.57e+03   2.9 2.89e+05    -  3.38e-03 7.77e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 280r 4.4385863e+04 8.91e+04 3.93e+03   2.9 2.96e+05    -  1.81e-03 1.42e-03f  1
 281r 4.6254448e+04 8.86e+04 9.10e+03   2.9 3.36e+05    -  5.91e-03 1.43e-03f  1
 282r 4.7146330e+04 8.84e+04 1.19e+04   2.9 3.93e+05    -  2.36e-03 5.97e-04f  1
 283r 5.0330511e+04 8.76e+04 1.28e+04   2.9 4.32e+05    -  2.90e-03 2.20e-03f  1
 284r 5.2680531e+04 8.69e+04 1.24e+04   2.9 4.12e+05    -  1.48e-03 1.56e-03f  1
 285r 5.4557031e+04 8.65e+04

 564  3.8898701e+05 6.84e+03 8.38e+03  -1.0 6.84e+03  -0.9 1.29e-03 7.60e-05h  1
 565  3.8899557e+05 6.84e+03 8.38e+03  -1.0 6.84e+03  -1.4 2.47e-04 1.92e-04h  1
 566  3.8899689e+05 6.84e+03 8.37e+03  -1.0 6.84e+03  -1.0 1.02e-03 2.87e-05h  1
 567  3.8900850e+05 6.84e+03 8.37e+03  -1.0 6.84e+03  -1.5 2.21e-04 2.73e-04h  1
 568r 3.8900850e+05 6.84e+03 9.99e+02   3.8 0.00e+00  -1.0 0.00e+00 3.93e-07R  5
 569r 3.8901480e+05 6.84e+03 1.01e+04   3.8 2.68e+06    -  8.72e-04 1.12e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 570r 4.1764777e+05 5.99e+02 1.46e+04   2.4 1.39e+06    -  1.15e-02 4.85e-03f  1
 571  4.1764943e+05 5.99e+02 8.99e+03  -1.0 5.99e+02  -1.5 1.17e-03 3.19e-04h  1
 572  4.1765042e+05 5.99e+02 8.99e+03  -1.0 5.99e+02  -1.1 1.09e-02 1.99e-04h  1
 573  4.1765137e+05 5.99e+02 8.98e+03  -1.0 7.11e+02  -1.6 7.34e-03 5.09e-04h  1
 574  4.1764938e+05 5.98e+02 8.98e+03  -1.0 2.08e+03  -2.1 6.25e-03 7.04e-04f  1
 575  4.1764796e+05 5.98e+02

 747  3.2571266e+05 3.29e+02 1.04e+03  -1.0 2.79e+04  -5.7 4.64e-04 1.30e-02f  1
 748  3.2568737e+05 3.27e+02 6.41e+02  -1.0 1.52e+04  -5.3 2.41e-01 6.30e-03f  1
 749  3.2495738e+05 2.69e+02 7.21e+02  -1.0 7.07e+03  -5.7 1.37e-03 1.64e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  3.2495328e+05 2.69e+02 6.07e+02  -1.0 8.61e+03  -5.3 9.77e-02 1.08e-03f  1
 751  3.2431551e+05 2.12e+02 4.80e+02  -1.0 5.27e+03  -4.9 3.63e-01 2.12e-01f  1
 752  3.2430933e+05 2.11e+02 4.89e+02  -1.0 6.02e+03  -5.4 2.08e-02 1.86e-03f  1
 753  3.2365899e+05 1.71e+02 3.83e+02  -1.0 5.22e+03  -4.9 7.71e-03 1.92e-01f  1
 754  3.2365756e+05 1.71e+02 3.77e+02  -1.0 1.21e+05  -5.4 3.14e-03 3.84e-05f  1
 755  3.2364895e+05 1.70e+02 3.96e+02  -1.0 5.86e+03  -5.0 2.03e-01 2.50e-03f  1
 756  3.2346648e+05 1.63e+02 3.40e+02  -1.0 5.89e+03  -5.5 1.15e-03 4.22e-02f  1
 757  3.2332911e+05 1.59e+02 3.24e+02  -1.0 1.11e+04  -5.1 6.89e-02 2.25e-02f  1
 758  3.2316686e+05 1.41e+02

 922  3.0062392e+05 5.25e+01 1.16e+02  -3.8 7.85e+03  -5.2 1.79e-03 3.62e-06f  1
 923  2.9976381e+05 4.75e+01 1.67e+02  -3.8 3.75e+03  -4.8 6.51e-05 2.38e-01f  1
 924  2.9976360e+05 4.75e+01 1.66e+02  -3.8 1.31e+04  -5.3 2.42e-03 1.77e-05f  1
 925  2.9918349e+05 4.12e+01 1.47e+02  -3.8 4.17e+03  -4.8 1.26e-01 1.41e-01f  1
 926  2.9918337e+05 4.12e+01 3.05e+02  -3.8 1.31e+04  -5.3 3.58e-02 9.49e-06f  1
 927  2.9875433e+05 3.77e+01 2.67e+02  -3.8 4.71e+03  -4.9 7.68e-05 9.19e-02f  1
 928  2.9875427e+05 3.77e+01 3.33e+02  -3.8 1.54e+04  -5.4 1.58e-02 3.91e-06f  1
 929  2.9774690e+05 3.17e+01 2.54e+02  -3.8 5.24e+03  -4.9 2.52e-05 1.83e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 930  2.9734036e+05 3.12e+01 2.47e+02  -3.8 2.55e+04  -5.4 1.71e-02 2.06e-02f  1
 931  2.9734021e+05 3.12e+01 2.48e+02  -3.8 6.04e+03  -5.0 5.93e-04 2.39e-05f  1
 932  2.9713209e+05 3.09e+01 3.29e+02  -3.8 1.95e+04  -5.5 2.09e-02 1.08e-02f  1
 933  2.9713206e+05 3.09e+01

...

1549  5.5082338e+03 1.15e+01 5.38e+02  -3.8 3.93e+02  -1.9 6.45e-04 2.49e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1550  5.5082222e+03 1.15e+01 5.27e+02  -3.8 9.48e+01  -2.4 4.30e-02 3.34e-05f  1
1551  5.5061801e+03 1.14e+01 5.26e+02  -3.8 1.08e+03    -  6.68e-04 7.20e-04f  1
1552  5.5051294e+03 1.14e+01 5.22e+02  -3.8 1.95e+02  -2.8 1.11e-02 1.41e-03f  1
1553  5.4745274e+03 1.13e+01 5.15e+02  -3.8 1.10e+03    -  3.31e-04 9.03e-03f  1
1554  5.4663155e+03 1.11e+01 5.02e+02  -3.8 9.87e+01  -2.4 2.73e-02 2.30e-02f  1
1555  5.4575879e+03 1.09e+01 4.98e+02  -3.8 3.30e+02  -2.9 7.01e-03 1.09e-02f  1
1556  5.4195659e+03 1.07e+01 4.91e+02  -3.8 4.20e+02  -3.4 6.98e-04 2.57e-02f  1
1557  5.4009385e+03 1.04e+01 4.83e+02  -3.8 2.16e+02  -2.9 1.39e-02 2.25e-02f  1
1558  5.3899706e+03 1.01e+01 4.55e+02  -3.8 1.11e+02  -2.5 7.14e-02 2.73e-02f  1
1559  5.3557852e+03 9.75e+00 4.44e+02  -3.8 2.37e+02  -3.0 1.70e-02 3.77e-02f  1
iter    objective    inf_pr 

1649  1.7473972e+03 8.72e-04 3.39e-01  -3.8 4.33e-01  -0.1 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1650  1.7420257e+03 8.11e-03 3.38e-01  -3.8 1.30e+00  -0.6 1.00e+00 1.00e+00f  1
1651  1.7267028e+03 7.20e-02 3.39e-01  -3.8 3.90e+00  -1.1 1.00e+00 9.48e-01f  1
1652  1.7161784e+03 9.79e-02 2.44e+00  -3.8 1.18e+01  -1.5 3.48e-01 2.15e-01f  1
1653  1.6978018e+03 1.26e-01 3.42e-01  -3.8 4.42e+00  -1.1 1.00e+00 1.00e+00f  1
1654  1.6809683e+03 2.20e-01 1.59e+01  -3.8 1.35e+01  -1.6 9.23e-01 3.00e-01f  1
1655  1.6086321e+03 4.18e+00 1.16e+01  -3.8 4.40e+01  -2.1 4.27e-01 4.09e-01f  1
1656  1.5886755e+03 3.48e+00 1.64e+02  -3.8 1.95e+01  -1.6 7.26e-01 2.66e-01f  1
1657  1.5886642e+03 3.48e+00 6.89e+02  -3.8 4.05e+01  -2.1 1.00e+00 6.70e-05f  1
1658  1.5088455e+03 3.82e+00 6.01e+02  -3.8 1.16e+02  -2.6 3.23e-01 1.69e-01f  1
1659  1.4108468e+03 3.46e+00 3.21e+02  -3.8 4.35e+01  -2.2 1.00e+00 5.50e-01f  1
iter    objective    inf_pr 

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1830  1.6305696e+02 8.78e-01 7.71e+02  -3.8 1.70e+03  -4.3 5.39e-04 4.30e-04f  1
1831  1.6271266e+02 8.76e-01 3.97e+02  -3.8 4.42e+02  -4.8 6.87e-02 3.38e-03f  1
1832  1.5380804e+02 1.07e+00 7.80e+02  -3.8 5.35e+02  -5.2 2.59e-05 5.99e-02f  1
1833  1.4326198e+02 1.51e+00 8.99e+02  -3.8 5.36e+02  -5.7 5.43e-02 8.13e-02f  1
1834  1.4284171e+02 1.51e+00 9.02e+02  -3.8 1.25e+03  -6.2 5.18e-06 7.96e-04f  1
1835  1.3647054e+02 1.67e+00 1.02e+03  -3.8 5.26e+02  -6.7 2.80e-02 5.48e-02f  1
1836  1.3558128e+02 1.66e+00 1.03e+03  -3.8 2.37e+04  -7.1 1.18e-06 1.99e-04f  1
1837  1.3186425e+02 1.70e+00 1.07e+03  -3.8 1.09e+03  -7.6 2.08e-02 3.42e-02f  1
1838  1.3149563e+02 1.70e+00 1.07e+03  -3.8 1.52e+05  -7.2 1.25e-07 1.91e-05f  1
1839  1.2895449e+02 1.71e+00 8.37e+02  -3.8 1.34e+03  -7.7 5.78e-02 2.39e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1840  1.2355701e+02 2.39e+00

1935  8.6567812e+01 4.10e-01 3.08e+01  -3.8 2.97e+00  -3.1 1.00e+00 2.15e-01h  1
1936  8.6486465e+01 3.42e-01 2.60e+01  -3.8 7.00e+00  -3.5 8.81e-01 1.66e-01h  1
1937  8.5686093e+01 1.55e-01 3.66e+01  -3.8 1.92e+01  -4.0 1.00e+00 5.55e-01h  1
1938  8.3238717e+01 1.42e-01 5.69e+01  -3.8 5.69e+01  -4.5 1.95e-01 5.67e-01f  1
1939  7.2278755e+01 5.11e+00 4.42e+02  -3.8 1.52e+02  -5.0 3.03e-01 9.66e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1940  7.1061522e+01 5.14e+00 4.62e+02  -3.8 4.66e+03    -  2.34e-02 4.31e-02f  1
1941  6.8037063e+01 5.43e+00 1.98e+02  -3.8 4.95e+02    -  4.29e-01 1.12e-01f  1
1942  6.0001646e+01 1.13e+01 1.02e+02  -3.8 5.27e+02    -  1.08e-01 3.08e-01f  1
1943  6.0000741e+01 1.13e+01 9.62e+01  -3.8 3.43e+04    -  4.72e-03 4.58e-05h  1
1944  5.9997554e+01 1.13e+01 9.38e+01  -3.8 1.68e+04    -  8.98e-03 8.99e-04h  1
1945  5.9993705e+01 1.11e+01 7.61e+01  -3.8 6.56e+02    -  1.65e-01 2.02e-02f  1
1946  5.9962566e+01 1.11e+01

2037  4.8027402e+01 2.54e-08 1.47e-02  -5.7 5.32e-02  -4.3 1.00e+00 1.00e+00h  1
2038  4.8027398e+01 1.23e-07 4.06e-01  -5.7 1.23e-01  -4.8 1.00e+00 1.00e+00h  1
2039  4.8027396e+01 1.00e-08 1.13e-01  -5.7 4.45e-02  -4.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2040  4.8027392e+01 9.00e-09 2.15e+01  -5.7 4.66e-01  -4.8 1.00e+00 2.17e-01h  3
2041  4.8027390e+01 8.90e-09 4.34e+01  -5.7 7.31e+00  -5.3 6.17e-02 1.04e-02h  3
2042  4.8027388e+01 5.38e-11 6.49e-03  -5.7 5.56e-03  -3.1 1.00e+00 1.00e+00h  1
2043  4.8027386e+01 1.76e-10 3.92e-02  -5.7 2.24e-02  -3.5 1.00e+00 1.00e+00h  1
2044  4.8027380e+01 6.61e-09 8.28e-01  -5.7 1.08e-01  -4.0 1.00e+00 1.00e+00h  1
2045  4.8027376e+01 7.92e-09 2.65e+01  -5.7 3.62e-01  -4.5 9.38e-01 1.27e-01h  2
2046  4.8027376e+01 5.59e-11 2.80e-03  -5.7 5.44e-03  -3.2 1.00e+00 1.00e+00h  1
2047  4.8027376e+01 1.51e-10 4.45e-04  -5.7 4.98e-03  -3.6 1.00e+00 1.00e+00h  1
2048  4.8027376e+01 8.88e-10

In [12]:
solve_pyomo(para_truth[8])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 197r 1.1127945e+04 9.72e+04 7.48e+01  -4.5 3.28e+01   0.1 4.83e-01 1.00e+00f  1
 198r 1.1217260e+04 9.72e+04 4.04e+02  -4.5 9.78e+01  -0.4 1.00e+00 2.95e-01f  1
 199r 1.1217292e+04 9.72e+04 8.41e+02  -4.5 2.89e+02  -0.9 2.33e-01 3.58e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200r 1.1905103e+04 9.70e+04 1.54e+03  -4.5 8.32e+02  -1.3 3.57e-01 2.61e-01f  1
 201r 1.2088459e+04 9.69e+04 1.08e+03  -4.5 3.19e+02  -0.9 7.78e-02 1.83e-01f  1
 202r 1.2160694e+04 9.69e+04 2.06e+03  -4.5 9.11e+02  -1.4 9.49e-02 2.47e-02f  1
 203r 1.2291935e+04 9.69e+04 1.82e+03  -4.5 3.56e+02  -1.0 1.15e-01 1.17e-01f  1
 204r 1.2563879e+04 9.68e+04 4.32e+02  -4.5 1.01e+03  -1.4 1.57e-02 8.32e-02f  1
 205r 1.2954953e+04 9.67e+04 3.53e+02  -4.5 3.95e+02  -1.0 3.23e-01 3.12e-01f  1
 206r 1.2954990e+04 9.67e+04 3.53e+02  -4.5 1.50e+02  -0.6 3.11e-05 7.62e-05f  1
 207r 1.3112302e+04 9.66e+04 6.99e+01  -4.5 5.65e+01  -0.2 1.00e+00 8.79e-01f  1
 208r 1.3179093e+04 9.66e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480  1.9948491e+05 2.74e+04 4.08e+03  -1.0 2.74e+04  -0.7 4.73e-05 4.02e-05h  1
 481  1.9949004e+05 2.74e+04 4.08e+03  -1.0 2.74e+04  -1.2 3.27e-05 2.14e-05h  1
 482  1.9949296e+05 2.74e+04 4.15e+03  -1.0 2.74e+04  -0.8 1.21e-04 1.15e-05h  1
 483  1.9950081e+05 2.74e+04 4.18e+03  -1.0 2.74e+04  -1.3 9.04e-05 4.13e-05h  1
 484r 1.9950081e+05 2.74e+04 9.99e+02   4.4 0.00e+00  -0.8 0.00e+00 4.98e-07R  5
 485r 1.9949946e+05 2.74e+04 3.28e+04   4.4 2.06e+07    -  7.32e-05 2.19e-08f  1
 486r 2.1480654e+05 2.18e+04 3.02e+04   2.3 1.24e+06    -  4.62e-03 4.60e-03f  1
 487  2.1481196e+05 2.18e+04 4.40e+03  -1.0 2.18e+04  -0.4 5.30e-06 2.24e-05h  1
 488r 2.1481196e+05 2.18e+04 9.99e+02   4.3 0.00e+00  -0.9 0.00e+00 3.04e-07R  6
 489r 2.1481239e+05 2.18e+04 1.69e+04   4.3 1.19e+08    -  2.04e-05 5.12e-09f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490r 2.2520966e+05 1.78e+04

 572  2.7332263e+05 9.35e+01 2.01e+05  -1.0 6.55e+02  -0.3 1.86e-03 2.81e-03h  1
 573  2.7332642e+05 9.32e+01 2.75e+05  -1.0 6.60e+02  -0.8 6.42e-02 3.14e-03h  1
 574  2.7333790e+05 9.19e+01 2.75e+05  -1.0 6.39e+02  -1.3 3.30e-02 1.39e-02h  1
 575  2.7334557e+05 9.12e+01 3.48e+05  -1.0 6.27e+02  -0.9 7.47e-02 8.18e-03h  1
 576  2.7335822e+05 8.97e+01 3.46e+05  -1.0 6.24e+02  -1.4 3.98e-02 1.56e-02h  1
 577  2.7336213e+05 8.94e+01 3.90e+05  -1.0 6.14e+02  -0.9 4.51e-02 4.19e-03h  1
 578  2.7337479e+05 8.80e+01 3.92e+05  -1.0 6.13e+02  -1.4 9.12e-02 1.54e-02h  1
 579  2.7338273e+05 8.72e+01 3.95e+05  -1.0 6.03e+02  -1.0 1.45e-02 8.89e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 580  2.7339753e+05 8.56e+01 3.90e+05  -1.0 5.97e+02  -1.5 6.55e-02 1.87e-02h  1
 581  2.7340293e+05 8.51e+01 4.18e+05  -1.0 5.86e+02  -1.0 3.84e-02 6.19e-03h  1
 582  2.7340887e+05 8.44e+01 4.25e+05  -1.0 5.82e+02  -1.5 1.52e-01 7.38e-03h  1
 583  2.7342530e+05 8.29e+01

 726  2.6047450e+05 2.09e+01 6.84e+05  -1.0 2.71e+03  -4.3 2.23e-02 5.80e-03f  1
 727  2.6034602e+05 2.02e+01 7.65e+05  -1.0 1.16e+03  -3.9 9.34e-03 3.20e-02f  1
 728  2.6022205e+05 1.87e+01 8.25e+05  -1.0 4.29e+02  -3.4 4.81e-02 7.41e-02f  1
 729  2.5984446e+05 1.78e+01 9.86e+05  -1.0 2.20e+03  -3.9 6.85e-03 5.42e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730  2.5890817e+05 1.13e+01 1.08e+06  -1.0 5.96e+02  -3.5 2.62e-01 3.65e-01f  1
 731  2.5881010e+05 1.12e+01 1.08e+06  -1.0 3.05e+03  -4.0 4.68e-03 1.10e-02f  1
 732  2.5877577e+05 1.12e+01 1.04e+06  -1.0 1.92e+04  -4.4 1.39e-02 1.03e-03f  1
 733  2.5857600e+05 9.14e+00 3.95e+05  -1.0 2.16e+02  -3.1 3.19e-01 1.82e-01f  1
 734  2.5855815e+05 8.73e+00 3.60e+05  -1.0 8.91e+01  -2.7 5.23e-02 4.53e-02f  1
 735  2.5844639e+05 7.95e+00 1.98e+05  -1.0 2.45e+02  -3.2 2.58e-01 8.91e-02f  1
 736  2.5785195e+05 6.79e+00 1.95e+05  -1.0 1.01e+03  -3.6 4.89e-03 1.46e-01f  1
 737  2.5780925e+05 6.60e+00

...

1898  1.3297555e+02 7.22e+00 3.23e+04  -5.7 7.52e+02    -  8.27e-04 5.24e-07h  1
1899  1.3297558e+02 7.22e+00 3.02e+04  -5.7 1.39e+02    -  8.69e-02 1.36e-06f 12
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1900  1.3297789e+02 7.21e+00 2.96e+04  -5.7 5.69e-01   1.6 2.78e-04 1.09e-03h  1
1901  1.3300860e+02 7.11e+00 2.92e+04  -5.7 6.78e-01   1.1 1.43e-02 1.44e-02h  1
1902  1.3303456e+02 7.02e+00 2.89e+04  -5.7 7.13e-01   0.6 1.44e-02 1.23e-02h  1
1903  1.3310457e+02 6.79e+00 2.77e+04  -5.7 7.25e-01   0.2 8.56e-02 3.35e-02h  1
1904  1.3313341e+02 6.69e+00 2.57e+04  -5.7 7.07e-01  -0.3 8.51e-02 1.42e-02h  1
1905  1.3323155e+02 6.36e+00 1.15e+03  -5.7 6.98e-01  -0.8 9.15e-01 4.91e-02h  1
1906  1.3329007e+02 6.17e+00 6.91e+02  -5.7 6.57e-01  -1.3 1.28e-01 3.08e-02h  1
1907  1.3381427e+02 4.41e+00 5.94e+02  -5.7 5.77e-01  -1.8 4.35e-01 2.85e-01h  1
1908  1.3406554e+02 3.56e+00 7.85e+02  -5.7 4.74e-01  -2.2 4.12e-01 1.94e-01h  1
1909  1.3434975e+02 2.52e+00

2061  9.9448870e+01 6.28e-05 3.41e+03  -5.7 1.02e-01   0.4 3.90e-01 4.93e-01f  1
2062  9.9437548e+01 5.15e-05 3.80e+02  -5.7 3.94e-02   0.8 1.00e+00 2.01e-01h  1
2063  9.9323653e+01 1.96e-04 4.13e+02  -5.7 1.53e-01   0.3 2.36e-01 8.08e-01h  1
2064  9.9216872e+01 4.43e-04 2.85e+02  -5.7 2.22e-01  -0.1 2.19e-01 3.39e-01h  1
2065  9.9182753e+01 3.67e-04 5.47e+01  -5.7 8.44e-02   0.3 4.23e-01 2.07e-01h  1
2066  9.8931083e+01 1.54e-03 3.03e+02  -5.7 3.30e-01  -0.2 1.82e-01 6.38e-01h  1
2067  9.8815000e+01 5.18e-04 2.33e+02  -5.7 1.54e-01   0.2 5.12e-01 1.00e+00h  1
2068  9.8751442e+01 6.86e-05 2.12e-01  -5.7 3.45e-02   0.7 1.00e+00 1.00e+00h  1
2069  9.8562384e+01 6.07e-04 1.38e+01  -5.7 1.76e-01   0.2 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2070  9.8530470e+01 4.70e-04 5.67e+02  -5.7 1.15e-01   0.6 1.00e+00 2.33e-01h  1
2071  9.8526882e+01 4.41e-04 3.73e+02  -5.7 2.98e-02   1.0 1.00e+00 6.21e-02h  1
2072  9.8444634e+01 1.08e-04

2231  8.5173185e+01 1.12e-05 3.38e+00  -5.7 2.56e+00  -4.3 1.00e+00 5.79e-01f  1
2232  8.5160956e+01 3.79e-03 1.29e-03  -5.7 7.67e+00  -4.8 1.00e+00 1.00e+00f  1
2233  8.5124991e+01 3.66e-02 1.10e-02  -5.7 2.30e+01  -5.2 1.00e+00 1.00e+00h  1
2234  8.5120200e+01 3.56e-02 1.85e+02  -5.7 6.90e+01  -5.7 1.00e+00 4.73e-02h  1
2235  8.5061631e+01 1.29e-01 3.85e+02  -5.7 2.07e+02  -6.2 1.00e+00 2.18e-01f  1
2236  8.4997168e+01 2.31e-01 9.04e+02  -5.7 6.25e+02  -6.7 1.00e+00 1.04e-01h  1
2237  8.4974698e+01 2.38e-01 1.06e+03  -5.7 1.41e+03  -7.1 2.56e-01 1.85e-02h  1
2238  8.4962643e+01 2.44e-01 1.05e+03  -5.7 1.26e+03  -7.6 1.34e-02 1.17e-02h  1
2239  8.4962639e+01 2.43e-01 2.55e+02  -5.7 7.32e-02   0.2 1.00e+00 8.19e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2240  8.4962756e+01 2.24e-01 9.16e+01  -5.7 1.15e-02  -0.2 6.40e-01 7.77e-02h  1
2241  8.4964171e+01 7.20e-09 4.55e+01  -5.7 2.24e-02  -0.7 5.03e-01 1.00e+00h  1
2242  8.4964168e+01 4.37e-11

2391  8.4421393e+01 1.49e-03 8.79e+00  -8.6 1.44e+01  -5.9 1.05e-04 7.33e-05h  1
2392  8.4421393e+01 1.49e-03 1.15e+01  -8.6 4.08e+01  -6.4 3.88e-03 2.19e-06f  1
2393  8.4421393e+01 1.49e-03 2.57e+02  -8.6 5.28e+02  -6.9 4.72e-02 2.87e-08h 13
2394  8.4417527e+01 1.57e-03 2.82e+02  -8.6 4.21e+02  -7.3 2.54e-01 1.06e-02f  1
In iteration 2394, 1 Slack too small, adjusting variable bound
2395  8.4417527e+01 1.57e-03 3.93e+02  -8.6 7.55e+02  -7.8 6.41e-02 3.83e-08h  1
2396  8.4417527e+01 1.57e-03 3.97e+02  -8.6 2.00e+03  -8.3 1.57e-03 6.06e-07h  1
2397  8.4417504e+01 1.57e-03 3.42e+02  -8.6 9.55e+03  -8.8 1.95e-02 3.75e-05f  1
2398  8.4404913e+01 5.93e-02 3.17e+02  -8.6 2.91e+03  -8.4 7.21e-02 2.77e-02f  1
2399  8.4358967e+01 2.11e-01 2.97e+02  -8.6 1.35e+03  -7.9 3.20e-02 4.10e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2400  8.4358965e+01 2.11e-01 2.09e+02  -8.6 8.30e-02  -0.1 1.94e-01 3.33e-04h  1
2401  8.4358966e+01 2.10e-01 1.70e+02  -8.6 1.

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2610  8.4144463e+01 2.02e-09 1.93e+00  -8.6 2.47e-01  -2.7 9.84e-02 7.10e-02f  1
2611  8.4144448e+01 2.38e-09 9.17e-01  -8.6 3.81e-01  -3.2 1.00e+00 4.98e-02f  1
2612  8.4144439e+01 2.42e-09 2.33e+00  -8.6 5.39e-01  -3.6 1.00e+00 3.58e-02f  1
2613  8.4144359e+01 3.89e-07 5.84e+00  -8.6 8.10e-01  -4.1 1.00e+00 2.46e-01f  1
2614  8.4144342e+01 4.51e-07 2.02e+01  -8.6 2.41e+00  -4.6 1.00e+00 1.91e-02f  1
2615  8.4144195e+01 1.61e-05 5.70e+01  -8.6 3.46e+00  -5.1 1.00e+00 8.14e-02f  1
2616  8.4144142e+01 1.80e-05 1.68e+02  -8.6 8.96e+00  -5.5 1.00e+00 1.09e-02h  1
2617  8.4143959e+01 4.35e-05 4.50e+02  -8.6 2.47e+01  -6.0 9.60e-01 1.38e-02h  1
2618  8.4143959e+01 4.34e-05 1.55e+00  -8.6 1.92e-04  -0.2 1.00e+00 3.18e-03h  1
2619  8.4143959e+01 1.52e-06 5.00e-02  -8.6 2.36e-04  -0.6 1.00e+00 9.30e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2620  8.4143959e+01 4.25e-07

In [13]:
solve_pyomo(para_truth[9])

Ipopt 3.14.6: max_iter=100000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality 

 284r 7.5127766e+04 8.08e+04 9.99e+02   4.9 0.00e+00   2.1 0.00e+00 9.85e-08R  2
 285r 7.5122872e+04 8.08e+04 2.89e+03   4.9 3.33e+07    -  2.41e-05 1.25e-07f  1
 286r 7.6254597e+04 8.05e+04 6.61e+03   2.8 5.37e+05    -  2.12e-03 5.47e-04f  1
 287r 8.2277590e+04 7.90e+04 4.97e+03   2.8 7.68e+05    -  8.92e-04 1.90e-03f  1
 288r 8.2651951e+04 7.90e+04 5.18e+03   2.8 8.99e+05    -  2.59e-04 1.01e-04f  1
 289r 8.3644489e+04 7.87e+04 5.68e+03   2.8 8.95e+05    -  5.96e-04 2.68e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 290r 8.5059296e+04 7.84e+04 6.30e+03   2.8 9.41e+05    -  7.88e-04 3.62e-04f  1
 291r 8.6895609e+04 7.79e+04 6.82e+03   2.8 9.69e+05    -  7.53e-04 4.53e-04f  1
 292r 8.9178454e+04 7.74e+04 8.09e+03   2.8 1.02e+06    -  1.20e-03 5.30e-04f  1
 293r 9.6965446e+04 7.56e+04 8.10e+03   2.8 1.09e+06    -  1.79e-03 1.66e-03f  1
 294r 1.0335359e+05 7.41e+04 9.30e+03   2.8 1.41e+06    -  1.72e-03 1.05e-03f  1
 295r 1.1341103e+05 7.18e+04

 386  3.4584333e+05 1.43e+04 6.33e+04  -1.0 3.17e+04  -3.7 1.82e-03 1.78e-04h  1
 387  3.4584603e+05 1.43e+04 6.50e+04  -1.0 2.93e+04  -3.3 2.99e-03 9.36e-05h  1
 388  3.4588780e+05 1.43e+04 6.50e+04  -1.0 3.12e+04  -3.8 1.57e-03 1.24e-03h  1
 389  3.4588843e+05 1.43e+04 8.02e+04  -1.0 2.99e+04  -3.4 2.81e-02 2.06e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390  3.4595072e+05 1.43e+04 8.00e+04  -1.0 3.10e+04  -3.8 1.26e-03 1.79e-03h  1
 391  3.4596005e+05 1.42e+04 8.00e+04  -1.0 3.15e+04  -4.3 1.76e-03 2.46e-04h  1
 392  3.4602247e+05 1.42e+04 8.04e+04  -1.0 3.07e+04  -3.9 4.81e-03 1.75e-03h  1
 393  3.4603282e+05 1.42e+04 8.05e+04  -1.0 3.16e+04  -4.4 4.53e-03 2.69e-04h  1
 394  3.4606045e+05 1.42e+04 8.05e+04  -1.0 7.50e+04    -  4.62e-04 6.61e-04h  1
 395r 3.4606045e+05 1.42e+04 9.99e+02   4.2 0.00e+00    -  0.00e+00 2.84e-07R  6
 396r 3.4606544e+05 1.42e+04 1.56e+03   4.2 1.92e+08    -  1.59e-05 6.80e-07f  1
 397r 3.5023127e+05 1.32e+04

 483  3.6635934e+05 2.26e+02 4.43e+04  -1.0 2.38e+04  -5.1 1.12e-01 5.72e-02f  1
 484  3.6409033e+05 2.03e+02 3.88e+04  -1.0 2.31e+04  -4.6 1.11e-01 1.01e-01f  1
 485  3.6277906e+05 1.94e+02 4.02e+04  -1.0 2.39e+04  -5.1 2.27e-02 4.51e-02f  1
 486  3.6084424e+05 1.77e+02 2.96e+04  -1.0 2.28e+04  -4.7 1.50e-01 8.65e-02f  1
 487  3.5862947e+05 1.65e+02 3.42e+04  -1.0 2.40e+04  -5.2 1.28e-02 6.74e-02f  1
 488  3.5521834e+05 1.42e+02 3.16e+04  -1.0 2.24e+04  -4.7 1.20e-01 1.42e-01f  1
 489  3.5520078e+05 1.42e+02 2.87e+04  -1.0 2.27e+04  -5.2 2.78e-02 7.13e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  3.5054825e+05 1.15e+02 3.49e+04  -1.0 2.20e+04  -4.8 5.67e-02 1.90e-01f  1
 491  3.5016931e+05 1.11e+02 3.39e+04  -1.0 2.03e+04  -4.4 2.92e-02 2.96e-02f  1
 492  3.5009704e+05 1.11e+02 2.22e+04  -1.0 2.14e+04  -4.8 1.46e-01 3.66e-03f  1
 493  3.4993020e+05 1.10e+02 2.16e+04  -1.0 2.03e+04  -4.4 2.12e-02 1.22e-02f  1
 494  3.3743104e+05 2.36e+02

 641  2.8009821e+05 4.81e+01 1.42e+04  -3.8 2.60e+03  -4.6 1.15e-04 9.11e-02f  1
 642  2.7823727e+05 6.74e+01 1.16e+04  -3.8 7.94e+03  -5.1 3.67e-04 1.84e-01f  1
 643  2.7823700e+05 6.74e+01 1.16e+04  -3.8 2.94e+03  -4.6 9.85e-01 8.21e-05f  1
 644  2.7822551e+05 6.71e+01 1.16e+04  -3.8 5.58e+03  -5.1 4.81e-04 4.67e-03f  1
 645  2.7821771e+05 6.67e+01 1.15e+04  -3.8 2.26e+03  -4.7 9.76e-04 6.06e-03f  1
 646  2.7819368e+05 6.66e+01 1.15e+04  -3.8 1.27e+04  -5.2 1.92e-04 1.80e-03f  1
 647  2.7817440e+05 6.63e+01 1.14e+04  -3.8 3.74e+03  -4.7 1.25e-01 4.50e-03f  1
 648  2.7817430e+05 6.63e+01 1.14e+04  -3.8 8.45e+03  -5.2 1.45e-04 9.15e-06f  1
 649  2.7769043e+05 6.22e+01 1.03e+04  -3.8 4.17e+03  -4.8 3.32e-02 9.70e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650  2.7756271e+05 6.18e+01 1.02e+04  -3.8 1.33e+04  -5.3 2.80e-03 7.71e-03f  1
 651  2.7756028e+05 6.18e+01 1.02e+04  -3.8 7.64e+03  -4.8 2.21e-04 4.03e-04f  1
 652  2.7699141e+05 6.31e+01

...

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1160  9.9143314e+02 6.77e+00 7.07e+02  -3.8 4.64e+02    -  6.74e-05 6.43e-05f  1
1161  9.8034130e+02 6.66e+00 7.73e+02  -3.8 4.68e+02    -  7.03e-02 1.58e-02f  1
1162  9.7298616e+02 6.59e+00 9.36e+02  -3.8 4.76e+02    -  1.48e-01 1.04e-02f  1
1163  9.7261936e+02 6.59e+00 1.03e+03  -3.8 5.06e+02    -  1.02e-01 5.04e-04f  1
1164  9.0437826e+02 6.00e+00 9.08e+02  -3.8 5.29e+02    -  5.44e-02 9.15e-02f  1
1165  8.5044610e+02 5.54e+00 8.38e+02  -3.8 4.87e+02    -  7.70e-02 7.73e-02f  1
1166  8.2175578e+02 5.30e+00 8.34e+02  -3.8 4.48e+02    -  1.14e-01 4.30e-02f  1
1167  7.6425254e+02 5.04e+00 7.60e+02  -3.8 4.02e+02    -  7.48e-02 8.63e-02f  1
1168  7.6422152e+02 5.03e+00 7.97e+02  -3.8 1.02e+01  -0.5 3.54e-05 1.64e-03h  1
1169  7.6422459e+02 5.03e+00 8.03e+02  -3.8 4.27e+01  -1.0 7.03e-04 2.74e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1170  7.6371374e+02 4.99e+00

1258  4.4775362e+02 9.63e-01 1.36e+03  -3.8 7.69e-01  -0.1 1.19e-01 1.94e-01f  1
1259  4.4690249e+02 8.98e-01 1.28e+03  -3.8 2.30e+00  -0.6 1.24e-01 6.80e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1260  4.4543786e+02 8.64e-01 1.24e+03  -3.8 6.85e+00  -1.1 6.53e-02 4.04e-02f  1
1261  4.4391696e+02 8.55e-01 1.22e+03  -3.8 2.05e+01  -1.6 2.83e-02 1.42e-02f  1
1262  4.4067957e+02 8.62e-01 1.21e+03  -3.8 6.09e+01  -2.0 1.46e-02 1.03e-02f  1
1263  4.3423831e+02 9.26e-01 1.20e+03  -3.8 1.79e+02  -2.5 7.16e-03 7.00e-03f  1
1264  4.3170695e+02 9.39e-01 1.20e+03  -3.8 1.10e+02  -2.1 3.86e-05 5.22e-03f  1
1265  4.3035427e+02 9.34e-01 1.20e+03  -3.8 2.85e+01  -1.7 4.70e-02 9.39e-03f  1
1266  4.2231126e+02 1.13e+00 1.18e+03  -3.8 1.43e+02  -2.1 3.71e-03 1.34e-02f  1
1267  4.2097261e+02 1.13e+00 1.18e+03  -3.8 3.72e+01  -1.7 4.44e-02 7.64e-03f  1
1268  4.2050771e+02 1.12e+00 1.17e+03  -3.8 1.13e+01  -1.3 3.65e-02 8.10e-03f  1
1269  4.1950042e+02 1.12e+00

1358  1.5709969e+02 2.21e+01 1.82e+03  -3.8 8.97e-01  -0.1 2.75e-03 9.47e-02h  1
1359  1.5706037e+02 2.13e+01 8.65e+02  -3.8 1.02e+00  -0.6 3.02e-01 3.54e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1360  1.5707063e+02 2.12e+01 1.07e+03  -3.8 6.73e-01   1.6 6.78e-03 4.15e-03h  1
1361  1.5708926e+02 2.11e+01 1.00e+03  -3.8 7.29e-01   1.1 4.24e-03 6.92e-03h  1
1362  1.5714701e+02 2.07e+01 8.22e+02  -3.8 6.92e-01   1.6 1.81e-02 2.08e-02h  1
1363  1.5714883e+02 2.06e+01 2.82e+03  -3.8 6.55e-01   2.0 1.29e-02 6.48e-04h  1
1364  1.5724847e+02 2.00e+01 1.13e+03  -3.8 7.04e-01   1.5 2.72e-03 3.29e-02h  1
1365  1.5726073e+02 1.99e+01 3.48e+03  -3.8 6.66e-01   1.9 1.88e-02 3.94e-03h  1
1366  1.5732794e+02 1.95e+01 3.33e+03  -3.8 6.88e-01   1.5 1.85e-02 2.01e-02h  1
1367  1.5732949e+02 1.95e+01 5.55e+03  -3.8 6.86e-01   1.9 1.69e-02 4.69e-04h  1
1368  1.5738619e+02 1.91e+01 5.83e+03  -3.8 6.96e-01   1.4 2.76e-02 1.76e-02h  1
1369  1.5750932e+02 1.84e+01

1527  4.8889089e+01 2.47e-02 6.72e+02  -3.8 2.68e+01  -0.9 2.97e-01 2.34e-03f  4
1528  4.8773109e+01 2.11e-02 4.09e+02  -3.8 1.36e+00  -1.4 1.00e+00 1.44e-01h  1
1529  4.8514413e+01 2.77e-02 4.00e+02  -3.8 9.09e+02    -  1.91e-02 2.03e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1530  4.8513222e+01 2.71e-02 2.29e+04  -3.8 6.84e-02   1.7 1.00e+00 2.40e-02h  1
1531  4.8491932e+01 2.64e-02 4.35e+03  -3.8 5.94e+01    -  8.09e-01 2.25e-02h  1
1532  4.8477748e+01 7.72e-02 4.34e+03  -3.8 4.65e+03    -  4.55e-03 1.84e-02h  1
1533  4.7460455e+01 3.40e-01 5.15e+03  -3.8 2.17e+02    -  6.23e-03 6.96e-01f  1
1534  4.7484132e+01 2.64e-01 4.08e+03  -3.8 2.72e-02   1.3 1.00e+00 2.21e-01h  1
1535  4.7570836e+01 9.94e-02 6.40e+00  -3.8 6.40e-02   0.8 1.00e+00 1.00e+00h  1
1536  4.7545815e+01 2.81e-01 6.47e+02  -3.8 3.37e-01   0.3 9.17e-01 2.31e-01h  1
1537  4.7433882e+01 4.48e-01 7.40e+02  -3.8 3.22e+03    -  3.20e-02 1.75e-02f  1
1538  4.7043968e+01 1.64e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1680  4.4775103e+01 2.44e-03 7.73e-01  -8.6 1.48e-02  -1.3 1.00e+00 1.82e-03h  1
1681  4.4775104e+01 2.14e-03 4.84e-03  -8.6 1.44e-04  -1.8 1.00e+00 1.25e-01f  4
1682  4.4775104e+01 3.78e-10 2.34e-05  -8.6 4.20e-03  -2.3 1.00e+00 1.00e+00h  1
1683  4.4775104e+01 1.46e-11 1.56e-06  -8.6 8.41e-04  -2.7 1.00e+00 1.00e+00h  1
1684  4.4775104e+01 1.46e-11 1.38e-06  -8.6 2.24e-03  -3.2 1.00e+00 1.00e+00h  1
1685  4.4775103e+01 4.37e-11 1.24e-03  -8.6 5.64e-03  -3.7 1.00e+00 6.91e-01h  1
1686  4.4775103e+01 1.02e-10 2.77e-03  -8.6 1.36e-02  -4.2 1.00e+00 7.17e-01f  1
1687  4.4775103e+01 2.08e-10 1.71e-02  -8.6 3.54e-02  -4.6 1.00e+00 3.12e-01f  1
1688  4.4775102e+01 1.46e-09 4.13e-02  -8.6 8.74e-02  -5.1 1.00e+00 3.47e-01f  1
1689  4.4775099e+01 1.21e-08 1.07e-01  -8.6 2.38e-01  -5.6 1.00e+00 3.81e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1690  4.4775094e+01 3.34e-08